# Designing oligos for half site library Sort-seq experiment 
What to first test the effect that the number of half-sites have on expresion
to do this I will need to make every posible combination of half-site mutations for a few promoters

Code written by PSR

In [1]:
from analyze_sig_hits_v2 import *


In [2]:
import itertools
import re
import random 
import pandas as pd
from collections import OrderedDict
from itertools import product
from Bio.Seq import *
from itertools import chain, combinations
from random import choice
from itertools import groupby
from operator import itemgetter  


In [3]:
def mutate_sequence_motif(sequence,orig_seq,mut_seq):
    #This function takes in a DNA sequence as a biopython Seq object and mutates the orig_seq to the mutated orig_seq
    #on both strands
    #it returns the sequence and the number of replacements
    return_seq = sequence.upper()
    forward_strand_str = str(return_seq) #
    count = forward_strand_str.count(str(orig_seq.upper()))
    #print(count)
    return_seq._data = forward_strand_str.replace(str(orig_seq.upper()),str(mut_seq.upper()))
    reverse_strand_str = str(return_seq.reverse_complement())
    count = count + reverse_strand_str.count(str(orig_seq.upper()))
    #print(count)
    return_seq._data = Seq(reverse_strand_str.replace(str(orig_seq.upper()),str(mut_seq.upper()))).reverse_complement()._data
    return return_seq, count

In [4]:
def find_substring_locations(string,substring):
    loc=0
    chopped = 0
    array = []
    current_length = len(string)
    loc = string.find("CGG")
    while (loc > -1) & (loc < len(string)):
        #record position of dectected string
        original_pos = loc+chopped
        array.append(original_pos)
        #chop off the first base of last string detected
        string = string[loc+1:]
        #record how much you chopped off
        chopped = chopped+loc+1
        #find the next instance of the string
        loc = string.find("CGG")
    return array

In [5]:

def block_start_positions(string,substring,forward,reverse):
    #this bit of code take a string and looks for all instances of a substring and returns 
    #an array of the indexes of positions corresponding to the substring, as well as 
    #a buffer of forward in front and reverse after.
    loc=0
    chopped = 0
    array = []
    current_length = len(string)
    loc = string.find(substring)
    while (loc > -1) & (loc < len(string)):
        #record position of dectected string
        original_pos = loc+chopped
        if (original_pos - forward) < 0:
            start = 0
        else:
            start = original_pos - forward
        end = original_pos + (len(substring)-1) + reverse
        for ind in range(start,end+1):
            array.append(ind)
        #chop off the first base of last string detected
        string = string[loc+1:]
        #record how much you chopped off
        chopped = chopped+loc+1
        #find the next instance of the string
        loc = string.find("CGG")
    array = list(OrderedDict.fromkeys(array)) #remove duplicate entries
    return array

In [6]:
def mutate_sequence_random(sequence,exclude_sequence,mut_sequence,number):
    #This function takes in a DNA ret_sequence as a biopython Seq object and mutates the string to the mutated string
    #on both strands
    ret_sequence = sequence.upper()
    exclude_sequence = exclude_sequence.upper()
    mut_sequence = mut_sequence.upper()
    valid_positions = []
    #this will be an array of sets.  [0] is the set of valid positions on the forward strand [1] on the reverse
    
    #find valid start positions for the mutated ret_sequence on the forward strand
    invalid_positions = block_start_positions(str(ret_sequence),str(exclude_sequence),len(mut_sequence)-1,0)
    #find positions that have the mutated seqeunce, we need to filter these, otherwise there is no mutation
    invalid_positions2 = block_start_positions(str(ret_sequence),str(mut_sequence),len(mut_sequence)-1,0)
    invalid_positions = invalid_positions + invalid_positions2
    valid_positions.append(set([x for x in range(0,len(ret_sequence)-len(mut_sequence)) if x not in invalid_positions]))
    
    #find valid start positions for the mutated ret_sequences on the reverse strand
    
    invalid_positions = block_start_positions(str(ret_sequence),str(exclude_sequence.reverse_complement()),0,len(mut_sequence)-1)
    invalid_positions2 = block_start_positions(str(ret_sequence),str(mut_sequence.reverse_complement()),len(mut_sequence)-1,0)
    valid_positions.append(set([x for x in range(0,len(ret_sequence)-len(mut_sequence)) if x not in invalid_positions]))
    
    #filter the valid positions to remove positions that have the mutated seqeunce, otherwise there is no mutation
    invalid_positions = block_start_positions(str(ret_sequence),str(exclude_sequence.reverse_complement()),0,len(mut_sequence)-1)
    
    #loop through and pick a strand, pick a location, make the mutation, pop the indexes from the valid starts
    #keep looping til done or no more valid starts on either strand
    
    for count in range(0,number):
        #have we exhausted all possible mutated sites?
        if (len(valid_positions[0])==0)&(len(valid_positions[1])==0):
            print "error:  not enough mutable sites"
            break
        elif len(valid_positions[0])==0: #if forward strand is fully mutated, mutate reverse strand
            mposition = random.choice(list(valid_positions[1])) #choose position
            temp = list(str(ret_sequence))
            temp[mposition:mposition+len(mut_sequence)] = list(str(mut_sequence.reverse_complement()))
            ret_sequence._data = "".join(temp)
            valid_positions[1].remove(mposition) #remove from list of valid positions
        elif len(valid_positions[1])==0:
            mposition = random.choice(list(valid_positions[0])) #choose position
            temp = list(str(ret_sequence))
            temp[mposition:mposition+len(mut_sequence)] = list(str(mut_sequence))
            ret_sequence._data = "".join(temp)
            valid_positions[0].remove(mposition) #remove from list of valid positions
        else:
            strand = random.choice([0,1])
            mposition = random.choice(list(valid_positions[0])) #choose position
            temp = list(str(ret_sequence))
            if strand:
                temp[mposition:mposition+len(mut_sequence)] = list(str(mut_sequence.reverse_complement()))
            else:
                temp[mposition:mposition+len(mut_sequence)] = list(str(mut_sequence))
            ret_sequence._data = "".join(temp)
            valid_positions[0].remove(mposition) #remove from list of valid positions
    return ret_sequence

In [7]:
def powerset(iterable):
    "powerset([1,2,3]) --> () (1,) (2,) (3,) (1,2) (1,3) (2,3) (1,2,3)"
    #iterates through a list and makes every possible combination 
    s = list(iterable)
    return chain.from_iterable(combinations(s, r) for r in range(len(s)+1))

In [8]:
def make_all_possible_halfsite_mut(ss):

    indices_start = [m.start() for m in re.finditer("CGG", ss)]
    R  = [m.start() for m in re.finditer("CCG", ss)]



    for index in indices_start:
        for p in R:
            pp = index - p
            if pp == 1:
                R.remove(p) 



    indices_start
    indices_start.extend(R)
    indices_start.sort()    

    to_replace = "CAA"
    sequence_list = []


    for replacements in powerset(indices_start):
            F_new_words = list(ss)
            for index in replacements:
                F_new_words[index:index+3] = to_replace
            sequence_list.append("".join(F_new_words))

    return sequence_list
    


In [9]:
# def check_for_site(df):
#     numb = []
#     for index, row in df.iterrows():
        
        
#         tf = row["tf"]
#         seq = row['sequence']
#         t = row['type']
#         num = row['halfsite_num']
#         tf = tf.replace("TF","")
#         n = row["Name"]
        
#         #print(tf)
        
#         tf = tf.replace('\xef',"")
#         tf =tf.replace('\xbb',"")
#         tf =tf.replace('\xbf',"")
#         m = get_max_pwm_score(seq, tf)
#         #print(m)
#         #motif = one_strong_or_two_weak(s,tf,m,dif=2)
#         motif = one_weak_one_strong(s, tf, m, dif=3)
#         print(motif)

In [10]:
def check_for_site(seq, tf):
    
        
#         tf = row["tf"]
#         seq = row['sequence']
#         t = row['type']
#         num = row['halfsite_num']
#         tf = tf.replace("TF","")
#         n = row["Name"]
        
#         #print(tf)
        
    tf = tf.replace('\xef',"")
    tf =tf.replace('\xbb',"")
    tf =tf.replace('\xbf',"")
    m = get_max_pwm_score(seq, tf)
    #print(m)
    #motif = one_strong_or_two_weak(s,tf,m,dif=2)
    motif = one_weak_one_strong(seq, tf, m, dif=3)
    print(motif)

Making controls:
1) Need to randomly mutate other sequences without conserving CG content Note: half-sites remain intact
2) Need to mutate half-sites while maintaing CG content ? 

In [11]:

#Works: randomly mutates mer sequences 
def random_mut_site(sequence, halfsite):
#this code randomly mutates 3mer sequences while maintaining half-sites

    seq1 = seq 

    ss , c = mutate_sequence_motif(Seq(seq1), "CGG", "PPP")

    indices_start_f = [m.start() for m in re.finditer("PPP", str(ss))]
    R  = [m.end() for m in re.finditer("PPP", str(ss))]



    list_indices = []
    for i in R:
        for e in indices_start_f:
            l = list(range(e,i))
            if len(l) == 3:
                for i in l:
                    list_indices.append(i)
                #list_of_indices.append(l)
    # indices_start_f.extend(R)
    # indices_start_f.sort()

    list_indices.extend(R)
    list_indices.sort()

    #print(list_indices)

    list_all_indices = []
    for i in range(len(seq)+1):
        list_all_indices.append(i)
    #print(list_all_indices)

    final_index = []
    for index in list_all_indices:
        if index not in list_indices:
            final_index.append(index)



    f_list = []
    for k, g in groupby(enumerate(final_index), lambda (i, x): i-x):
        p = (map(itemgetter(1), g))  
        if len(p) >= 3:
            lst = p[:len(p)-2]
            for i in lst:
                f_list.append(i)
    #print(f_list)            



    final_index_list= random.sample(f_list, halfsite)


    to_replace = "CAA"
    sequence_list = []


    for replacements in powerset(final_index_list):
            F_new_words = list(seq)
            for index in replacements:
                F_new_words[index:index+3] = to_replace
            sequence_list.append("".join(F_new_words))
            
    


    return sequence_list




In [12]:
def random_mut_count(sequence, halfsite):
#this is used to count the number half-sites 

    seq1 = seq 

    ss , c = mutate_sequence_motif(Seq(seq1), "CGG", "PPP")

    indices_start_f = [m.start() for m in re.finditer("PPP", str(ss))]
    R  = [m.end() for m in re.finditer("PPP", str(ss))]



    list_indices = []
    for i in R:
        for e in indices_start_f:
            l = list(range(e,i))
            if len(l) == 3:
                for i in l:
                    list_indices.append(i)
                #list_of_indices.append(l)
    # indices_start_f.extend(R)
    # indices_start_f.sort()

    list_indices.extend(R)
    list_indices.sort()

    #print(list_indices)

    list_all_indices = []
    for i in range(len(seq)+1):
        list_all_indices.append(i)
    #print(list_all_indices)

    final_index = []
    for index in list_all_indices:
        if index not in list_indices:
            final_index.append(index)



    f_list = []
    for k, g in groupby(enumerate(final_index), lambda (i, x): i-x):
        p = (map(itemgetter(1), g))  
        if len(p) >= 3:
            lst = p[:len(p)-2]
            for i in lst:
                f_list.append(i)
    #print(f_list)            



    final_index_list= random.sample(f_list, halfsite)


    to_replace = "PPP"
    sequence_list = []


    for replacements in powerset(final_index_list):
            F_new_words = list(seq)
            for index in replacements:
                F_new_words[index:index+3] = to_replace
            sequence_list.append("".join(F_new_words))
            
  


    return sequence_list



In [13]:
#this control works it adds CGs to sequence that have had their half-sites removed 

def remove_site_maintain_CG(sequence):
    
    len_seq = len(sequence)
    CG_bases = 170 - len(sequence) 
    ss_list = list(sequence)
    list_int1 = []
    
    for i in range(len(ss_list)):
        if ss_list[i] == "A" or ss_list[i] == "T":
            list_int1.append(i)
            
    list_int1 = list_int1[:55]
    last_i = len(sequence)-1


    for num in list_int1:
        if num == 0 or num == len_seq:
            list_int1.remove(num)
            
    #print(len(list_int1))


    list_ind = random.sample(list_int1, CG_bases) 
    #print(len(list_ind))



    first_index = [0]
    last_index = [len(seq)+1]
    #lists are combined and sorted smallest to largest
    list_ind.extend(last_index)
    list_ind.extend(first_index)
    list_ind.sort()

  

    final_index_list = []



    # #uses the indices to split the sequence into "words"
    [final_index_list.append(x) for x in list_ind if x not in final_index_list] 
    #print(len(final_index_list))
    parts = [sequence[i:j] for i,j in zip(final_index_list, final_index_list[1:])]
    #creates indices where there isnt a half-site 
    site = [i for i, e in enumerate(parts) ]
    site.remove(0)


    # will add a half-site to indices without a site and returns a new sequence 
    lista = ['C'] * len(site)
    #print(len(lista))
    assert(len(lista)) == len(site)
    acc = 0
    for i in range(len(lista)):
        parts.insert(site[i]+acc, lista[i])
        acc += 1
    new_seq = "".join(parts)
    
#     for s in new_seq:
#         motif = check_for_site(s, TF)

#         if motif == 1:
#             break
#         else: 
#             return new_seq


    
    
    
    return new_seq



In [14]:
def remove_site(seq):

    new_sequences = [] 
    seq2 = seq
    lens = len(seq)
    to_replace = "PPP"

    indices_start = [m.start() for m in re.finditer("CGG", seq)]
    R  = [m.start() for m in re.finditer("CCG", seq)]

    for index in indices_start:
            for p in R:
                pp = index - p
                if pp == 1:
                    R.remove(p) 





    indices_start.extend(R)
    indices_start.sort()    



    seq_list  = []
    c = []
    for replacements in powerset(indices_start):
            F_new_words = list(seq)
            for index in replacements:
                F_new_words[index:index+3] = to_replace
            mut_list = "".join(F_new_words)
            #seq_list.append(mut_list)
            fm ,cc  = mutate_sequence_motif(Seq(mut_list),"PPP","")
            #print(len(fm))
            seq_list.append(str(fm))
            f ,count  = mutate_sequence_motif(fm,"CGG","CAA")
            c.append(count)

            
    df_mut = pd.DataFrame(list(zip(seq_list, c)), columns =['sequence', 'halfsite_num']) 
    
    d = df_mut.loc[df_mut['halfsite_num'] == 1]
    max_num = max(c)
    if d.empty:
        d = df_mut.loc[df_mut['halfsite_num'] == 1]
        
        df_no_one = []
        for i in range(2, max_num+1):
            df_f = df_mut.loc[df_mut['halfsite_num'] == i].sample(1, random_state=1)
            df_no_one.append(df_f)

        no_one = pd.concat(df_no_one)
        return no_one 
    else:
        newd = []
        for i in range(1, max_num+1):
            df_f = df_mut.loc[df_mut['halfsite_num'] == i].sample(1, random_state=1)
            newd.append(df_f)

        new = pd.concat(newd)
        return new 


# Making all possible combinations of half-site mutations 


In [15]:
df = pd.read_csv('Lib_control_seq_final2.csv',  header= None , sep=',')
df.columns = ["TF","name", "sequence"]
df_seq = df[~df['name'].str.contains("control")]


In [16]:
final_list = []
TF_list = []
name_list = []
type_mut = []
add = "site_mutation"
for index, row in df_seq.iterrows():
    tf_n = row["TF"]
    seq = row["sequence"]
    n = row["name"]
    combo = make_all_possible_halfsite_mut(seq)
    for i in combo:
        final_list.append(i)
        name = n
        name_list.append(name)
        type_mut.append(add)
        TF_list.append(tf_n)

    
   

In [17]:
cc = []
for seq in final_list:
    s, c = mutate_sequence_motif(Seq(seq), "CGG", "CAA")
    cc.append(c)

In [18]:

df_mut = pd.DataFrame(list(zip(TF_list ,type_mut, name_list, final_list, cc)), columns =['tf','type','Name', 'sequence', 'halfsite_num'])
df_mut

,tf,type,Name,sequence,halfsite_num
0,GAL4,site_mutation,ADR1pr_reg1,GTACGGCATCCGCGCGGCTCCGATCGCCCGTCCTACTTTTGCGGCC...,15
1,GAL4,site_mutation,ADR1pr_reg1,GTACAACATCCGCGCGGCTCCGATCGCCCGTCCTACTTTTGCGGCC...,14
2,GAL4,site_mutation,ADR1pr_reg1,GTACGGCATCAACGCGGCTCCGATCGCCCGTCCTACTTTTGCGGCC...,14
3,GAL4,site_mutation,ADR1pr_reg1,GTACGGCATCCGCGCAACTCCGATCGCCCGTCCTACTTTTGCGGCC...,14
4,GAL4,site_mutation,ADR1pr_reg1,GTACGGCATCCGCGCGGCTCAAATCGCCCGTCCTACTTTTGCGGCC...,14
5,GAL4,site_mutation,ADR1pr_reg1,GTACGGCATCCGCGCGGCTCCGATCGCCAATCCTACTTTTGCGGCC...,14
6,GAL4,site_mutation,ADR1pr_reg1,GTACGGCATCCGCGCGGCTCCGATCGCCCGTCCTACTTTTGCAACC...,14
7,GAL4,site_mutation,ADR1pr_reg1,GTACGGCATCCGCGCGGCTCCGATCGCCCGTCCTACTTTTGCGGCC...,14
8,GAL4,site_mutation,ADR1pr_reg1,GTACGGCATCCGCGCGGCTCCGATCGCCCGTCCTACTTTTGCGGCC...,14
9,GAL4,site_mutation,ADR1pr_reg1,GTACGGCATCCGCGCGGCTCCGATCGCCCGTCCTACTTTTGCGGCC...,14


# Controls

In [19]:
#randomly mutates 3mer
randsite_mut_list = []
randsite_TF_list = []
randsite_name_list = []
control_type = []
random_CGG_count = []
add = "random_site_mutation"
for index, row in df_seq.iterrows():
    tf_n = row["TF"]
    seq = row["sequence"]
    final_s , count = mutate_sequence_motif(Seq(seq),"CGG","CAA")
    n = row["name"]
    combo = random_mut_site(final_s, count)
    for_count = random_mut_count(final_s,count)
    #print(combo)
    for i in combo:
        randsite_mut_list.append(i)
        name = n
        randsite_name_list.append(name)
        control_type.append(add)
        randsite_TF_list.append(tf_n)
    for l in for_count:
        seq , cc= mutate_sequence_motif(Seq(l),"PPP", "CAA")
        random_CGG_count.append(cc)
        

In [20]:
#removes half-site but maintains CG content
removesite_CG_list = []
TF_list = []
name_list = []
control2_type = []
add = "site_mutation_CG_maintain"
df_removed_sites = []
for index, row in df_seq.iterrows():
    
    seq = row["sequence"]
    n = row["name"]
#     t = row['tf']
    combo = remove_site(seq)
 
    df_removed_sites.append(combo)
#     print(t)
    
#     TF_list.append(tf_n)
       


In [21]:
# df_removed_sites

In [22]:
half_site = []
CG_control = []
new_df = pd.concat(df_removed_sites)
for index, row in new_df.iterrows():
    s = row['sequence']
    #t = row['tf']
    r = remove_site_maintain_CG(s)
    CG_control.append(r)
    ss , c= mutate_sequence_motif(Seq(r), "CGG","CAA")
    half_site.append(c)
    
    
    #print(ss, c)
    #print(r)
    

In [23]:
CG_control_df = pd.DataFrame(list(zip(CG_control, half_site)), columns =['sequence', 'halfsite_num'])
#CG_control_df.to_csv("CG_control.csv") 

In [24]:
randsite_mut_df = pd.DataFrame(list(zip(randsite_TF_list ,control_type, randsite_name_list, randsite_mut_list, random_CGG_count)), columns =['tf','type','Name', 'sequence', 'halfsite_num'])

In [25]:
randsite_mut_df
#randsite_mut_df.to_csv("randsite_mut.csv")

,tf,type,Name,sequence,halfsite_num
0,GAL4,random_site_mutation,ADR1pr_reg1,GTACGGCATCCGCGCGGCTCCGATCGCCCGTCCTACTTTTGCGGCC...,0
1,GAL4,random_site_mutation,ADR1pr_reg1,GTACGGCATCCGCGCGGCTCCGATCGCCCGTCCTACTTTTGCGGCC...,1
2,GAL4,random_site_mutation,ADR1pr_reg1,GTACGGCATCCGCGCGGCTCCGATCGCCCGTCCTACTTTTGCGGCC...,1
3,GAL4,random_site_mutation,ADR1pr_reg1,GTACGGCATCCGCGCGGCTCCGATCGCCCGTCCTACTTTTGCGGCC...,1
4,GAL4,random_site_mutation,ADR1pr_reg1,GTACGGCATCCGCGCGGCTCCGATCGCCCGTCCTACCAATGCGGCC...,1
5,GAL4,random_site_mutation,ADR1pr_reg1,GTACGGCATCCGCGCGGCTCCGATCGCCCGTCCTACTTTTGCGGCC...,1
6,GAL4,random_site_mutation,ADR1pr_reg1,GTACGGCATCCGCGCGGCTCCGATCGCCCGTCCTACTTTTGCGGCC...,1
7,GAL4,random_site_mutation,ADR1pr_reg1,GTACGGCATCCGCGCGGCTCCGATCGCCCGTCCTACTTTTGCGGCC...,1
8,GAL4,random_site_mutation,ADR1pr_reg1,GTACGGCATCCGCGCGGCTCCGATCGCCCGTCCTCAATTTGCGGCC...,1
9,GAL4,random_site_mutation,ADR1pr_reg1,GTACGGCATCCGCGCGGCTCCGATCGCCCGTCCTACTTTTGCGGCC...,1


Will sample from each df

In [26]:
Lys = df_mut[df_mut["Name"].str.contains('LYS')]

In [27]:
no_lys = df_mut[~df_mut["Name"].str.contains("LYS")] 
no_lys
#df_seq = df_seq[~df_seq['name'].str.contains("GAL1")]
df_seq2= df_seq[~df_seq["name"].str.contains("LYS")] 
name = df_seq2["name"]
all_names = df_seq["name"]

In [28]:
#use this for every possible combination of mutated half-site
def sample_mut_seq(name, df_name, sample_num):
#this samples 10 sequences from every group of half-site number 1- max 
    df_mut2 = df_name[df_name['Name'].str.contains(name)]
    df_max = df_mut2.loc[df_mut2['halfsite_num'] == max(df_mut2['halfsite_num'])]
    df_max_minus1 = df_mut2.loc[df_mut2['halfsite_num'] == max(df_mut2['halfsite_num']-1)]
    df_min = df_mut2.loc[df_mut2['halfsite_num'] == min(df_mut2['halfsite_num'])]
    df_min_plus1 = df_mut2.loc[df_mut2['halfsite_num'] == min(df_mut2['halfsite_num']+1)]#.sample(n=10, random_state=1)
    max_num = max(df_mut2['halfsite_num']-1)    
    newd = []
    for i in range(2, max_num):
        df_f = df_mut2.loc[df_mut2['halfsite_num'] == i].sample(sample_num, random_state=1)
        newd.append(df_f)
    new = pd.concat(newd)
    newest = [df_min, df_min_plus1, new, df_max_minus1, df_max]
    result = pd.concat(newest)
    #final_df = pd.concat(result)
    return result


In [29]:
def sample_mut_seq_control(name, df_name, sample_num):
#this is used for smaller sampling i.e 1 sequence for the controls 
    df_mut2 = df_name[df_name['Name'].str.contains(name)]
    max_num = max(df_mut2['halfsite_num'])  
    newd = []
    for i in range(1, max_num+1):
        df_f = df_mut2.loc[df_mut2['halfsite_num'] == i].sample(sample_num, random_state=1)
        newd.append(df_f)
    new = pd.concat(newd)
    return new



In [30]:
final = []
for n in name:
    #print(n)
    f = sample_mut_seq(n,no_lys,5)
    final.append(f)

In [31]:
final_mut_df = pd.concat(final)

In [32]:
complete = [final_mut_df, Lys]
final_sampled_mut = pd.concat(complete)
final_sampled_mut

,tf,type,Name,sequence,halfsite_num
32767,GAL4,site_mutation,ADR1pr_reg1,GTACAACATCAACGCAACTCAAATCGCCAATCCTACTTTTGCAACC...,0
32752,GAL4,site_mutation,ADR1pr_reg1,GTACAACATCAACGCAACTCAAATCGCCAATCCTACTTTTGCAACC...,1
32753,GAL4,site_mutation,ADR1pr_reg1,GTACAACATCAACGCAACTCAAATCGCCAATCCTACTTTTGCAACC...,1
32754,GAL4,site_mutation,ADR1pr_reg1,GTACAACATCAACGCAACTCAAATCGCCAATCCTACTTTTGCAACC...,1
32755,GAL4,site_mutation,ADR1pr_reg1,GTACAACATCAACGCAACTCAAATCGCCAATCCTACTTTTGCAACC...,1
32756,GAL4,site_mutation,ADR1pr_reg1,GTACAACATCAACGCAACTCAAATCGCCAATCCTACTTTTGCAACC...,1
32757,GAL4,site_mutation,ADR1pr_reg1,GTACAACATCAACGCAACTCAAATCGCCAATCCTACTTTTGCAACC...,1
32758,GAL4,site_mutation,ADR1pr_reg1,GTACAACATCAACGCAACTCAAATCGCCAATCCTACTTTTGCAACC...,1
32759,GAL4,site_mutation,ADR1pr_reg1,GTACAACATCAACGCAACTCAAATCGCCAATCCTACTTTTGCAACC...,1
32760,GAL4,site_mutation,ADR1pr_reg1,GTACAACATCAACGCAACTCAAATCGCCAATCCTACTTTTGCAACC...,1


In [33]:
#final_sampled_mut.to_csv("final_sitemut_sampled.csv") 

In [34]:
rand2 = randsite_mut_df[~randsite_mut_df["Name"].str.contains("LYS")] 
#rand2
filtered_rand = []
for n in all_names:
    #print(n)
    f =  sample_mut_seq_control(n, randsite_mut_df, 1)
    filtered_rand.append(f)
    

In [35]:
fr = pd.concat(filtered_rand)
fr
# new_seq = []
# for index, row in df_mut.iterrows(): 
#     name = row["Name"]
#     seq = row["seq"]
#     num = row['halfsite_num']

#     if name 
    

,tf,type,Name,sequence,halfsite_num
4,GAL4,random_site_mutation,ADR1pr_reg1,GTACGGCATCCGCGCGGCTCCGATCGCCCGTCCTACCAATGCGGCC...,1
351,GAL4,random_site_mutation,ADR1pr_reg1,GTACGGCATCCGCGCGGCTCCGATCGCCCGTCCTACTTCAACGGCC...,2
1892,GAL4,random_site_mutation,ADR1pr_reg1,GTACGGCATCCGCGCGGCTCCGATCGCCCGTCCTCAATCAACGGCC...,3
1213,GAL4,random_site_mutation,ADR1pr_reg1,GTACGGCATCCGCGCGGCTCCGATCGCCCGTCCTACTTTTGCGGCC...,4
6543,GAL4,random_site_mutation,ADR1pr_reg1,GTACGGCATCCGCGCGGCTCCGATCGCCCGTCCTACTTCAACGGCC...,5
6194,GAL4,random_site_mutation,ADR1pr_reg1,GTACGGCATCCGCGCGGCTCCGATCGCCCGTCCTCAAAATGCGGCC...,6
18364,GAL4,random_site_mutation,ADR1pr_reg1,GTACGGCATCCGCGCGGCTCCGATCGCCCGTCCTACCACAACGGCC...,7
24199,GAL4,random_site_mutation,ADR1pr_reg1,GTACGGCATCCGCGCGGCTCCGATCGCCCGTCCTACCACAACGGCC...,8
29736,GAL4,random_site_mutation,ADR1pr_reg1,GTACGGCATCCGCGCGGCTCCGATCGCCCGTCCTCAAACAACGGCC...,9
28323,GAL4,random_site_mutation,ADR1pr_reg1,GTACGGCATCCGCGCGGCTCCGATCGCCCGTCCTACTTTTGCGGCC...,10


# To determine if the orientation of the half-site has an effect on expression
will change the orientation for promoter sequences to the opposite orientation of half-site sequences

In [47]:
dic1 = {"CCG":"CRR","CGG":"CCG"}
dic2 = {"CRR": "CGG"}

In [48]:
# dicF = {"CCG":"CGG"}
# dicR = {"CGG":"CCG"}

In [49]:
def replace_all(text, dic):
    #this function replaces all CGG half-sites with its reverse complement 
    for i, j in dic.items():
        text = text.replace(i, j)
    return text

In [50]:
def change_site_orientation(sequence, dic1, dic2):
    #this function changes the orientation of every single half-site in sequence 
    new_seq = replace_all(sequence, dic1)   
    change = replace_all(new_seq, dic2)


    return change 
    

In [65]:
def orientation_face_same_way(sequence):
    seq, count = mutate_sequence_motif(Seq(sequence),"CGG","P")
    new_seq = str(seq)
    indices_end = [m.start() for m in re.finditer("P", new_seq)]
    seq2, count = mutate_sequence_motif(Seq(new_seq),"P","")
    final = str(seq2)
    seq3, count = mutate_sequence_motif(Seq(final),"CGG","CAA")
    finalseq = str(seq3)
 
    forward_index = indices_end[0::2]
    reverse_index = indices_end[1::2]

    char_list = list(finalseq)


    list_f = ["CGG"] * len(forward_index)
    list_r = ["CCG"] * len(reverse_index)



    for i in range(len(list_r)):
        char_list.insert(forward_index[i], list_f[i])
        char_list.insert(reverse_index[i], list_r[i])
        
        
    new_seq = "".join(char_list)
        
#     for s in new_seq:

#         motif = check_for_site(s, TF)

#         if motif == 1:
#             break
#         else: 
#             return new_seq


    return "".join(char_list)

In [66]:
#assign random orientation 
#sequence = 'CGGxxxCGGxxCCGxxxCGGxxxxCGGxCCGxxxCCGxx'

def space_orientation(sequence):
    seq, count = mutate_sequence_motif(Seq(sequence),"CGG","P")
    new_seq = str(seq)
    indices = [m.start() for m in re.finditer("P", new_seq)]
    seq2, count2 = mutate_sequence_motif(Seq(new_seq),"P","")
    final = str(seq2)
    seq3, count3 = mutate_sequence_motif(Seq(final),"CGG","CAA")
    finalseq = str(seq3)
    char_list = list(finalseq)

    

    half_site = ["CGG", "CCG"]

    add_half = []

    for i in range(1,count):
        random_half = random.choice(half_site)
        add_half.append(random_half)

    for i in range(len(add_half)):
        char_list.insert(indices[i], add_half[i])
        
#     new_seq = "".join(char_list)
        
#     for s in new_seq:

#         motif = check_for_site(s, TF)

#         if motif == 1:
#             break
#         else: 
#             return new_seq

    return "".join(char_list)

    


In [67]:
sequence = 'CGGxxxCGGxxCCGxxxCGGxxxxCGGxCCGxxxCCGxx'
change_site_orientation(sequence, dic1, dic2)

'CCGxxxCCGxxCGGxxxCCGxxxxCCGxCGGxxxCGGxx'

# Changing the orientations of zinc cluster sequences
I will be making 14 oligos per promoter the first with all half-sites in the forward orientation (CGG) and the second with all half-sites in reverse orientation (CCG) . The third will be the opposite orientation of the WT sequence. The fourth will have the half-sites facing towards eachother and away from eachother. Seqences 5-9 will be spacing and randomized orientation combined. The sequences will be named:


|Name|
|----|
|Gene_pr_orientationF  |
|Gene_pr_orientationR  |
|orientation_opposite |
|orientation_face |
|orientation_|


In [68]:
df_orientation = pd.read_csv('Lib_control_seq_final.csv',  header= None , sep=',')
df_orientation.columns = ["tf","name", "sequence"]
df_orientation

,tf,name,sequence
0,﻿GAL4,ADR1pr_reg1,GTACGGCATCCGCGCGGCTCCGATCGCCCGTCCTACTTTTGCGGCC...
1,GAL4,HAP4pr_reg1,CCCGGGCCCCGGAGGTTGATCGCACGGCAAATACATAAGAACCTTC...
2,GAL4,STP4pr_reg1,CCTCCGGCTGCTTTTTAGCCTTCTACGCGCGGGACGGCTTGCGCGC...
3,GAL4,GID8pr_reg1,TCGGCTTGGGTTTGTGTGTACGGCCCTTTTTGGGATCCCGGAGTTT...
4,GAL4,GAL1_10pr,CCCCATTATCTTAGCCTAAAAAAACCTTCTCTTTGGAACTTTCAGT...
5,YRM1,YRM1_Mpr20pr,CCGCGGGATGCCATGCACGGATGCCTGCGGAGACAGTGGCGCGGAG...
6,YRM1,YRM1_Mnn4pr,GTGCCGTAGTCTGTATACGGTTTTAGATGCGGTATGTTTATCATCG...
7,YRM1,YRM1_Tpo4/Mod5pr,CGGAGCCCCTGCAGCTTCATGAAGACGGAACCGCACGGGCCTATTA...
8,YRM1,YRM1_control_Hsv2pr,TGCACTGTTACCATAGGATCTTACCAATTTGCATTATTTCCGCCAA...
9,YRM1,YRM1_control_Cax4pr,AGAAGCTCCGCTGATTATCCGTGGAAAAGGGTCTTATTCTGCCGTA...


In [70]:
v1 = "orientationF"
v2 = "orientationR"
v = "orientationfacesame"
v3 = "orientationopposite"

orientation_names = []
sequences = []
mutype = []
tf_name = []
orientation_names_face = []
sequences_face = []
mutype_face = []
tf_name_face = []
orientation_opposite = []
sequences_opposite = []
mutype_opposite = []
tf_name_opposite = []

for index, row in df_orientation.iterrows(): 
    seq = Seq(row["sequence"])
    n = row["name"]
    tf = row["tf"]
    [forward_orientation, num] = mutate_sequence_motif(seq, "CGG", "CCG")
    #count.append(num)
    sequences.append(str(forward_orientation))
    nameF = n
    orientation_names.append(nameF) 
    mutype.append(v1)
    tf_name.append(tf)
    [reverse_orientation, num] = mutate_sequence_motif(seq, "CCG", "CGG")
    #count.append(num)
    sequences.append(str(reverse_orientation))
    nameR = n
    orientation_names.append(nameR)
    mutype.append(v2)
    tf_name.append(tf)
    o_forward = orientation_face_same_way(str(seq))
    #print(o_forward)
    sequences_face.append(o_forward)
    orientation_names_face.append(n)
    mutype_face.append(v)
    tf_name_face.append(tf)
    opposite = change_site_orientation(str(seq), dic1, dic2)
    sequences_opposite.append(opposite)
    orientation_opposite.append(n)
    mutype_opposite.append(v3)
    tf_name_opposite.append(tf)
    

In [71]:
df_orientation_fr = pd.DataFrame(list(zip(tf_name, mutype ,orientation_names, sequences)), columns =['tf','type','Name', 'sequence'])
df_orientation_face = pd.DataFrame(list(zip(tf_name_face, mutype_face ,orientation_names_face, sequences_face)), columns =['tf','type','Name', 'sequence'])
df_orientation_opposite = pd.DataFrame(list(zip(tf_name_opposite, mutype_opposite ,orientation_opposite, sequences_opposite)), columns =['tf','type','Name', 'sequence'])
fin  = [df_orientation_fr, df_orientation_face, df_orientation_opposite]
df_orientation_fin  = pd.concat(fin)
#df_orientation_face



fin_counts = []
for index, row in df_orientation_fin.iterrows(): 
    sequence = row["sequence"]
    #print(sequence)
    s = Seq(sequence)
    all_seq , num = mutate_sequence_motif(s, "CGG", "CAA")
    
    
    fin_counts.append(num)
    
df_orientation_fin['halfsite_num'] = fin_counts
df_orientation_fin

,tf,type,Name,sequence,halfsite_num
0,﻿GAL4,orientationF,ADR1pr_reg1,GTACGGCATCGGCGCGGCTCGGATCGCCGGTCCTACTTTTGCGGCC...,15
1,﻿GAL4,orientationR,ADR1pr_reg1,GTACCGCATCCGCGCCGCTCCGATCGCCCGTCCTACTTTTGCCGCC...,15
2,GAL4,orientationF,HAP4pr_reg1,CCCGGGCCCCGGAGGTTGATCGCACGGCAAATACATAAGAACCTTC...,11
3,GAL4,orientationR,HAP4pr_reg1,CCCGGGCCCCGGAGGTTGATCGCACCGCAAATACATAAGAACCTTC...,11
4,GAL4,orientationF,STP4pr_reg1,CCTCCGGCTGCTTTTTAGCCTTCTACGCGCGGGACGGCTTGCGCGC...,10
5,GAL4,orientationR,STP4pr_reg1,CCTCCGGCTGCTTTTTAGCCTTCTACGCGCCGGACCGCTTGCGCGC...,10
6,GAL4,orientationF,GID8pr_reg1,TCGGCTTGGGTTTGTGTGTACGGCCCTTTTTGGGATCCCGGAGTTT...,9
7,GAL4,orientationR,GID8pr_reg1,TCCGCTTGGGTTTGTGTGTACCGCCCTTTTTGGGATCCCGGAGTTT...,9
8,GAL4,orientationF,GAL1_10pr,CCCCATTATCTTAGCCTAAAAAAACCTTCTCTTTGGAACTTTCAGT...,8
9,GAL4,orientationR,GAL1_10pr,CCCCATTATCTTAGCCTAAAAAAACCTTCTCTTTGGAACTTTCAGT...,8


# How does half-sites compare to a full conensus site?

a) How many half-sites do you need to get the same level of expression as a full consensus site?
b) Does evolution use half-sites to tune expression?

In [72]:
#officially works -.- 

def halfsite_compareA(sequence, halfsite_num, TF):
#this adds removes consensus sites 
    sequen, count = mutate_sequence_motif(Seq(sequence),"CGG","")
    seq1 = str(sequen)
    sequ, count = mutate_sequence_motif(Seq(seq1),"CGG","CAA")
    seq = str(sequ)
    len_seq = len(seq)
    #print(len_seq)
   
    number_insert = 3 * halfsite_num
   
    #print(number_insert)
    if len_seq + number_insert > 170:
        numb = (len_seq + number_insert) - 170
        n = len_seq-numb
        final_seq = seq[:n]
        last_i = len(final_seq)-1
        random_indices = random.sample(range(1,last_i,3), (halfsite_num)) 
        for index in random_indices:
            if index == 0 or index == len_seq:
                random_indices.remove(index)

        first_index = [0]
        last_index = [len(final_seq)+1]
        #lists are combined and sorted smallest to largest
        random_indices.extend(last_index)
        random_indices.extend(first_index)
        
        
        #mylist = ["a", "b", "a", "c", "c"]
        #random_indices = list( dict.fromkeys(random_indices) )
        
        random_indices.sort()
        
        
        final_index_list = []
      
        # #uses the indices to split the sequence into "words"
        [final_index_list.append(x) for x in random_indices if x not in final_index_list] 
        #print(final_index_list)
        #print(final_index_list)
        parts = [final_seq[i:j] for i,j in zip(final_index_list, final_index_list[1:])]
        #creates indices where there isnt a half-site 
        site = [i for i, e in enumerate(parts) if e != "CGG"]
        site.remove(0)
        #will add a half-site to indices without a site and returns a new sequence 
        lista = ['CGG'] * len(site)
        assert(len(lista)) == len(site)
        acc = 0
        for i in range(len(lista)):
            parts.insert(site[i]+acc, lista[i])
            acc += 1
        
        new_seqs = "".join(parts)
        
        for s in new_seqs:
        
            motif = check_for_site(s, TF)

            if motif == 1:
                break
            else: 
                return new_seqs

        
        
        

#         b = [sequence[i:i+17] for i in xrange(len(new_seqs))]
#         for seq in b:
#             s, c = mutate_sequence_motif(Seq(seq), "CGG", "XYZ")
#             new_seq = str(s)
#             indices = [m.start() for m in re.finditer("XYZ", new_seq)]
#             if 0 in indices and 14 in indices:
#                 break
#             else: 
#                 return "".join(parts)
#         return "".join(parts)

    
    
    else:
        
        last_i= len(seq)-1
        random_i = random.sample(range(1,last_i,3), halfsite_num)
#     # #     adds the first and last index
        last= [len(seq)+1]
        first = [0]
        #lists are combined and sorted smallest to largest
        random_i.extend(last)
        random_i.extend(first)
        random_i.sort()
        #print(random_indices)
        final_index = []
        #print(random_indices)
        # #uses the indices to split the sequence into "words"
        [final_index.append(x) for x in random_i if x not in final_index] 
        seq_parts = [seq[i:j] for i,j in zip(final_index, final_index[1:])]
        #creates indices where there isnt a half-site 
        site = [i for i, e in enumerate(seq_parts)]
        site.remove(0)
        #will add a half-site to indices without a site and returns a new sequence 
        lista = ['CGG'] * len(site)
        assert(len(lista)) == len(site)
        acc = 0
        for i in range(len(lista)):
            seq_parts.insert(site[i]+acc, lista[i])
            acc += 1
        
        ns = "".join(seq_parts)
        
        for seq in ns:
            motif_2 = check_for_site(seq, TF)

            if motif_2 == 1:
                break
            else: 
                return ns
            
#         bleep = [sequence[i:i+17] for i in xrange(len(ns))]
#         for seq in bleep:
#             s, c = mutate_sequence_motif(Seq(seq), "CGG", "XYZ")
#             ns = str(s)
#             indices = [m.start() for m in re.finditer("XYZ", ns)]
#             if 0 in indices and 14 in indices:
#                 break
#             else: 
#                 return "".join(seq_parts)            


        
        
    
    

In [73]:
df = pd.read_csv('Lib_control_seq_final2.csv',  header= None , sep=',')
df.columns = ["tf","name", "sequence"]
#df_control = df[~df['name'].str.contains("RGT")]
df_control = df[df['name'].str.contains("control")]

df_control


,tf,name,sequence
4,GAL4,GAL7pr_control,GCCAGTCCTTCGGATCACGGTCAACAGTTGTCCGAGCGCTTTTTGG...
5,GAL4,RUP1/SFL1pr_control,CCGGCTTCTTCTCTTCTTTCCTCTACTCCGACAGGTCCTTAAGCCT...
6,GAL4,GAL2pr_control,CGAAAGGGGCGGTTGCCTCAGGAAGGCACCGGCGGTCTTTCGTCCG...
7,GAL4,GAL1_10pr_control,CCCCATTATCTTAGCCTAAAAAAACCTTCTCTTTGGAACTTTCAGT...
11,YRM1,YRM1_control_Hsv2pr,TGCACTGTTACCATAGGATCTTACCAATTTGCATTATTTCCGCCAA...
12,YRM1,YRM1_GIP3/ISU1pr_control,CACTGAGACGTAATGTGGAAACACCACTTCTCGGTCCGCGGACATC...
13,YRM1,YRM1_Spo24pr_control,CATGCTTTTTCATGACGCCTCCTCCCCGCCCCTGAGCCTTCGGTGG...
14,YRM1,YRM1_control_Cax4pr,AGAAGCTCCGCTGATTATCCGTGGAAAAGGGTCTTATTCTGCCGTA...
18,TEA,TEA1_control_Isf1pr,AAGAAATCGGTAAAAGTGTAATTGTATTTATAATAGTGAGTTTCGG...
19,TEA,TEA1_control_Adh3pr,AGGGCATTTGTCGAGGGTAATACTGCATACTGCCTATTGTCGGTCC...


In [74]:
insert = "insert_site"
insertsite_seq = []
insertsite_names = []
mut_type = []
tf_n = []
for i in range(1,27, 2):
    for index, row in df_control.iterrows(): 
        seq = row["sequence"]
        n = row["name"]
        tf = row["tf"]
        insertsite = halfsite_compareA(seq, i,tf)
        insertsite_seq.append(insertsite)
        name = n 
        insertsite_names.append(name)
        mut_type.append(insert)
        tf_n.append(tf)
        

<analyze_sig_hits_v2.pssm instance at 0x2ae0d7484ab8>
0
<analyze_sig_hits_v2.pssm instance at 0x2ae0d7484200>
0
<analyze_sig_hits_v2.pssm instance at 0x2ae0d7484cb0>
0
<analyze_sig_hits_v2.pssm instance at 0x2ae0d7484200>
0
<analyze_sig_hits_v2.pssm instance at 0x2ae0d7484ab8>
0
<analyze_sig_hits_v2.pssm instance at 0x2ae0d74846c8>
0
<analyze_sig_hits_v2.pssm instance at 0x2ae0d7484cb0>
0
<analyze_sig_hits_v2.pssm instance at 0x2ae0d7484200>
0
<analyze_sig_hits_v2.pssm instance at 0x2ae0d7484ab8>
0
<analyze_sig_hits_v2.pssm instance at 0x2ae0d74846c8>
0
<analyze_sig_hits_v2.pssm instance at 0x2ae0d7484cb0>
0
<analyze_sig_hits_v2.pssm instance at 0x2ae0d7484320>
0
<analyze_sig_hits_v2.pssm instance at 0x2ae0d7484ab8>
0
<analyze_sig_hits_v2.pssm instance at 0x2ae0d7484320>
0
<analyze_sig_hits_v2.pssm instance at 0x2ae0d7484ab8>
0
<analyze_sig_hits_v2.pssm instance at 0x2ae0d74846c8>
0
<analyze_sig_hits_v2.pssm instance at 0x2ae0d7484cb0>
0
<analyze_sig_hits_v2.pssm instance at 0x2ae0d748

0
<analyze_sig_hits_v2.pssm instance at 0x2ae0d75116c8>
0
<analyze_sig_hits_v2.pssm instance at 0x2ae0d7511878>
0
<analyze_sig_hits_v2.pssm instance at 0x2ae0d75116c8>
0
<analyze_sig_hits_v2.pssm instance at 0x2ae0d7511878>
0
<analyze_sig_hits_v2.pssm instance at 0x2ae0d7511758>
0
<analyze_sig_hits_v2.pssm instance at 0x2ae0d7511878>
0
<analyze_sig_hits_v2.pssm instance at 0x2ae0d7511758>
0
<analyze_sig_hits_v2.pssm instance at 0x2ae0d7511878>
0
<analyze_sig_hits_v2.pssm instance at 0x2ae0d7511758>
0
<analyze_sig_hits_v2.pssm instance at 0x2ae0d7511878>
0
<analyze_sig_hits_v2.pssm instance at 0x2ae0d7511758>
0
<analyze_sig_hits_v2.pssm instance at 0x2ae0d7511878>
0
<analyze_sig_hits_v2.pssm instance at 0x2ae0d7511758>
0
<analyze_sig_hits_v2.pssm instance at 0x2ae0d7511c20>
0
<analyze_sig_hits_v2.pssm instance at 0x2ae0d7511758>
0
<analyze_sig_hits_v2.pssm instance at 0x2ae0d7511c20>
0
<analyze_sig_hits_v2.pssm instance at 0x2ae0d7511758>
0
<analyze_sig_hits_v2.pssm instance at 0x2ae0d7

In [75]:
df_insertsite = pd.DataFrame(list(zip(tf_n,mut_type ,insertsite_names, insertsite_seq)), columns =['tf','type','Name', 'sequence']) 

In [76]:
#df_insertsite.to_csv("final_insertsite.csv") 

# Controls

Need to make controls
1)add C/G bases to keep GC content the same without the half-sites 
Note: have to include a check to make sure I am not adding GAL4 sites
2) do random inserts without caring about CG content 


In [77]:
#sequence ="GTACGGCATCCGCGCGGCTCCGATCGCCCGTCCTACTTTTGCGGCCCGGCGCCGACGGCCTCCCCGGAATTTACCAACGAAGAGGAAAATAAACTGGAGTTATAACGCCATTGGCGGGAAACTCCGAAACCATAGCGCCGTGGTCTTTCCCCGGTTTTTTGGCGCCGGGT"

In [78]:
def make_insert_controls_1(sequence, halfsite_num, TF):
    
    CG_bases = halfsite_num * 3
    num_bases_add = int((CG_bases/2)-1)
    #print(num_bases_add)
    
    seq1, count = mutate_sequence_motif(Seq(sequence),"CGG","")
    seq2, count = mutate_sequence_motif(seq1 ,"CGG","CAA")
    seq = str(seq2)
    len_seq = len(seq)
    ss_list = list(seq)
    list_int1 = []
    list_int2 = []
    for i in range(len(ss_list)):
        if ss_list[i] == "A" or ss_list[i] == "T":
            list_int1.append(i)
    
    list_int1 = list_int1[:55]
    if len_seq + CG_bases > 170:
        numb = (len_seq + CG_bases) - 170
        n = len_seq-numb
        final_seq = seq[:n]
        last_i = len(final_seq)-1
        
        list_ind = random.sample(list_int1, num_bases_add) 
        #print(list_ind)

        for index in list_ind:
            if index == 0 or index == len_seq:
                list_ind.remove(index)

        

        first_index = [0]
        last_index = [len(seq)+1]
        #lists are combined and sorted smallest to largest
        list_ind.extend(last_index)
        list_ind.extend(first_index)
        list_ind.sort()
        
        #print(list_ind)


        final_index_list = []
        


        # #uses the indices to split the sequence into "words"
        [final_index_list.append(x) for x in list_ind if x not in final_index_list] 
        #print(final_index_list)
        parts = [final_seq[i:j] for i,j in zip(final_index_list, final_index_list[1:])]
        #creates indices where there isnt a half-site 
        site = [i for i, e in enumerate(parts) ]
        site.remove(0)
        

        # will add a half-site to indices without a site and returns a new sequence 
        lista = ['CC'] * len(site)
        assert(len(lista)) == len(site)
        acc = 0
        for i in range(len(lista)):
            parts.insert(site[i]+acc, lista[i])
            acc += 1
        new_seq = "".join(parts)
        
        
        for s in new_seq:
            
            motif = check_for_site(s, TF)

            if motif == 1:
                break
            else: 
                return new_seq
        
        
        #return new_seq
    
    else:
        
        #last_index = len(final_seq)-1

        new_list = random.sample(list_int1, num_bases_add) 

        for index in new_list:
            if index == 0 or index == len_seq:
                new_list.remove(index)



        first_index = [0]
        last_index = [len(seq)+1]
        #lists are combined and sorted smallest to largest
        new_list.extend(last_index)
        new_list.extend(first_index)
        new_list.sort()


        final_index_list = []


        # #uses the indices to split the sequence into "words"
        [final_index_list.append(x) for x in new_list if x not in final_index_list] 
        parts = [seq[i:j] for i,j in zip(final_index_list, final_index_list[1:])]
        #creates indices where there isnt a half-site 
        site = [i for i, e in enumerate(parts) ]
        site.remove(0)
        
        lista = ['CC'] * len(site)
        assert(len(lista)) == len(site)
        acc = 0
        for i in range(len(lista)):
            parts.insert(site[i]+acc, lista[i])
            acc += 1
        short_seq = "".join(parts)
        
        for s in short_seq:
        
            motif = check_for_site(s, TF)

            if motif == 1:
                break
            else: 
                return short_seq
        
        
        #return short_seq


In [79]:
#make_insert_controls_1(sequence, 25)

In [80]:
import itertools
s='ATCG'
list_3_mer = []
for seq in itertools.product('ATCG', repeat =3):
    list_seq = ''.join(seq)
    if "CC" in list_seq or "GG" in list_seq or "CG"  in list_seq or "GC" in list_seq or "CAC" in list_seq or "CTC" in list_seq or "GAG" in list_seq or "GTG" in list_seq or "GCG" in list_seq or "CGC" in list_seq:
        next
    else:
        list_3_mer.append(list_seq)

#print(list_3_mer)

In [81]:
def insert_random_3mer(sequence, halfsite_num, TF):
    
    seq1, count = mutate_sequence_motif(Seq(sequence),"CGG","")
    seq2, count = mutate_sequence_motif(seq1 ,"CGG","CAA")
    seq = str(seq2)
    len_seq = len(seq)
    ss_list = list(seq)
    number_insert = 3 * halfsite_num
    
    list_int1 = []
    
    for i in range(len(ss_list)):
        if ss_list[i] == "A" or ss_list[i] == "T":
            list_int1.append(i)
    
    list_int1 = list_int1[:50]
    

    if len_seq + number_insert > 170:
            numb = (len_seq + number_insert) - 170
            n = len_seq-numb
            final_seq = seq[:n]
            last_i = len(final_seq)-1
            random_indices = random.sample(list_int1, halfsite_num) 
            #random_indices = random.sample(range(1,last_i,3), (halfsite_num)) 
            for index in random_indices:
                if index == 0 or index == len_seq:
                    random_indices.remove(index)

            first_index = [0]
            last_index = [len(final_seq)+1]

            random_indices.extend(last_index)
            random_indices.extend(first_index)



            random_indices.sort()


            final_index_list = []


            [final_index_list.append(x) for x in random_indices if x not in final_index_list] 
            #print(final_index_list)
            #print(final_index_list)
            parts = [final_seq[i:j] for i,j in zip(final_index_list, final_index_list[1:])]
            #creates indices where there isnt a half-site 
            site = [i for i, e in enumerate(parts) if e != "CGG"]
            site.remove(0)


            lista = random.sample(list_3_mer, len(site))
            #print(lista)

            #will add a half-site to indices without a site and returns a new sequence 
            #lista = ['CGG'] * len(site)
            assert(len(lista)) == len(site)
            acc = 0
            for i in range(len(lista)):
                parts.insert(site[i]+acc, lista[i])
                acc += 1

            new_seqs = "".join(parts)

#             b = [sequence[i:i+17] for i in xrange(len(new_seqs))]
#             for seq in b:
#                 s, c = mutate_sequence_motif(Seq(seq), "CGG", "XYZ")
#                 new_seq = str(s)
#                 indices = [m.start() for m in re.finditer("XYZ", new_seq)]
#                 if 0 in indices and 14 in indices:
#                     break
#                 else: 




            for s in new_seqs:
        
                motif = check_for_site(s, TF)

                if motif == 1:
                    break
                else: 
                    return new_seqs    
            #return "".join(parts)

    else:

            last_i= len(seq)-1
            random_i = random.sample(range(1,last_i,3), halfsite_num)
    #     # #     adds the first and last index
            last= [len(seq)+1]
            first = [0]
            #lists are combined and sorted smallest to largest
            random_i.extend(last)
            random_i.extend(first)
            random_i.sort()
            #print(random_indices)
            final_index = []
            #print(random_indices)
            # #uses the indices to split the sequence into "words"
            [final_index.append(x) for x in random_i if x not in final_index] 
            seq_parts = [seq[i:j] for i,j in zip(final_index, final_index[1:])]
            #creates indices where there isnt a half-site 
            site = [i for i, e in enumerate(seq_parts)]
            site.remove(0)
            #will add a half-site to indices without a site and returns a new sequence 

            lista = random.sample(list_3_mer, len(site))
            #lista = ['CGG'] * len(site)
            assert(len(lista)) == len(site)
            acc = 0
            for i in range(len(lista)):
                seq_parts.insert(site[i]+acc, lista[i])
                acc += 1

            ns = "".join(seq_parts)

#             bleep = [sequence[i:i+17] for i in xrange(len(ns))]
#             for seq in bleep:
#                 s, c = mutate_sequence_motif(Seq(seq), "CGG", "XYZ")
#                 ns = str(s)
#                 indices = [m.start() for m in re.finditer("XYZ", ns)]
#                 if 0 in indices and 14 in indices:
#                     break
#                 else: 


            for s in ns:
        
                motif = check_for_site(s, TF)

                if motif == 1:
                    break
                else: 
                    return ns

            #return "".join(seq_parts)            

In [82]:
#seq = "GTACGGCATCCGCGCGGCTCCGATCGCCCGTCCTACTTTTGCGGCCCGGCGCCGACGGCCTCCCCGGAATTTACCAACGAAGAGGAAAATAAACTGGAGTTATAACGCCATTGGCGGGAAACTCCGAAACCATAGCGCCGTGGTCTTTCCCCGGTTTTTTGGCGCCGGGT"

In [83]:
# for i in range(1,27, 2):
#     p= make_insert_controls_1(seq, i)
#     s, n = mutate_sequence_motif(Seq(p), "CGG", "GTG")
#     print(n)
# #     ss, nn = mutate_sequence_motif(s, "CGG", 'C')
# #     print(nn)
    

In [84]:
insert_CG_1 = "insert_CG_control"
insert_CG_seq = []
insert_CG_names = []
mut_type_CG_control = []
tf_n_CG_control = []

insert_3mer = 'insert_rand_3mer'
insert_3mer_seq = []
insert_3mer_names = []
mut_type_3mer_control = []
tf_3mer = []

for i in range(1,27, 4):
    for index, row in df_control.iterrows(): 
        seq = row["sequence"]
        n = row["name"]
        tf = row["tf"]
        insertsite = make_insert_controls_1(seq, i, tf)
        insert_CG_seq.append(insertsite)
        name = n 
        insert_CG_names.append(name)
        mut_type_CG_control.append(insert_CG_1)
        tf_n_CG_control.append(tf)
        rand_seq = insert_random_3mer(seq,i, tf)
        rs , c= mutate_sequence_motif(Seq(rand_seq),"CGG", "GTG")
        insert_3mer_seq.append(str(rs))
        insert_3mer_names.append(n)
        mut_type_3mer_control.append(insert_3mer)
        tf_3mer.append(tf)
        
        

<analyze_sig_hits_v2.pssm instance at 0x2ae0d7482e18>
0
<analyze_sig_hits_v2.pssm instance at 0x2ae0d7482cb0>
0
<analyze_sig_hits_v2.pssm instance at 0x2ae0d7482b00>
0
<analyze_sig_hits_v2.pssm instance at 0x2ae0d7482bd8>
0
<analyze_sig_hits_v2.pssm instance at 0x2ae0d7482878>
0
<analyze_sig_hits_v2.pssm instance at 0x2ae0d7482bd8>
0
<analyze_sig_hits_v2.pssm instance at 0x2ae0d7482b90>
0
<analyze_sig_hits_v2.pssm instance at 0x2ae0d7482128>
0
<analyze_sig_hits_v2.pssm instance at 0x2ae0d7482a70>
0
<analyze_sig_hits_v2.pssm instance at 0x2ae0d7482998>
0
<analyze_sig_hits_v2.pssm instance at 0x2ae0d74822d8>
0
<analyze_sig_hits_v2.pssm instance at 0x2ae0d74821b8>
0
<analyze_sig_hits_v2.pssm instance at 0x2ae0d7482248>
0
<analyze_sig_hits_v2.pssm instance at 0x2ae0d7482248>
0
<analyze_sig_hits_v2.pssm instance at 0x2ae0d74820e0>
0
<analyze_sig_hits_v2.pssm instance at 0x2ae0d7482170>
0
<analyze_sig_hits_v2.pssm instance at 0x2ae0d74827a0>
0
<analyze_sig_hits_v2.pssm instance at 0x2ae0d748

<analyze_sig_hits_v2.pssm instance at 0x2ae0d77cb6c8>
0
<analyze_sig_hits_v2.pssm instance at 0x2ae0d77cb518>
0
<analyze_sig_hits_v2.pssm instance at 0x2ae0d77cb518>
0
<analyze_sig_hits_v2.pssm instance at 0x2ae0d77cb518>
0
<analyze_sig_hits_v2.pssm instance at 0x2ae0d77cb758>
0
<analyze_sig_hits_v2.pssm instance at 0x2ae0d77cb758>
0
<analyze_sig_hits_v2.pssm instance at 0x2ae0d77cb0e0>
0
<analyze_sig_hits_v2.pssm instance at 0x2ae0d77cb2d8>
0
<analyze_sig_hits_v2.pssm instance at 0x2ae0d77cb2d8>
0
<analyze_sig_hits_v2.pssm instance at 0x2ae0d77cb2d8>
0
<analyze_sig_hits_v2.pssm instance at 0x2ae0d77cb518>
0
<analyze_sig_hits_v2.pssm instance at 0x2ae0d77cbdd0>
0
<analyze_sig_hits_v2.pssm instance at 0x2ae0d77cb878>
0
<analyze_sig_hits_v2.pssm instance at 0x2ae0d77cb878>
0
<analyze_sig_hits_v2.pssm instance at 0x2ae0d77cbdd0>
0
<analyze_sig_hits_v2.pssm instance at 0x2ae0d77cb878>
0
<analyze_sig_hits_v2.pssm instance at 0x2ae0d77cb2d8>
0
<analyze_sig_hits_v2.pssm instance at 0x2ae0d77c

In [85]:
df_insert_CG = pd.DataFrame(list(zip(tf_n_CG_control,mut_type_CG_control ,insert_CG_names, insert_CG_seq)), columns =['tf','type','Name', 'sequence'])
df_insert_3mer = pd.DataFrame(list(zip(tf_3mer, mut_type_3mer_control ,insert_3mer_names, insert_3mer_seq)), columns =['tf','type','Name', 'sequence'])
df_insert_CG

,tf,type,Name,sequence
0,GAL4,insert_CG_control,GAL7pr_control,GCCAGTCCTTATCATCAACAGTTGTAGCGCTTTTTGGACCCTTTCC...
1,GAL4,insert_CG_control,RUP1/SFL1pr_control,CCTTCTTCTCTTCTTTCCTCTACTACAGGTCCTTAAGCCTTGTAGT...
2,GAL4,insert_CG_control,GAL2pr_control,CGAAAGGGGTTGCCTCAGGAAGGCACTCTTTCGTTGAGATATCTGC...
3,GAL4,insert_CG_control,GAL1_10pr_control,CCCCATTATCTTAGCCTAAAAAAACCTTCTCTTTGGAACTTTCAGT...
4,YRM1,insert_CG_control,YRM1_control_Hsv2pr,TGCACTGTTACCATAGGATCTTACCAATTTGCATTATTTCCAACTA...
5,YRM1,insert_CG_control,YRM1_GIP3/ISU1pr_control,CACTGAGACGTAATGTGGAAACACCACTTCTTACATCAAAAGGAAA...
6,YRM1,insert_CG_control,YRM1_Spo24pr_control,CATGCTTTTTCATGACGCCTCCTCCCCCCTGAGCCTTTGGAAGTGT...
7,YRM1,insert_CG_control,YRM1_control_Cax4pr,AGAAGCTCTGATTATTGGAAAAGGGTCTTATTCTGTAGGAAACTTC...
8,TEA,insert_CG_control,TEA1_control_Isf1pr,AAGAAATTAAAAGTGTAATTGTATTTATAATAGTGAGTTTAACTTA...
9,TEA,insert_CG_control,TEA1_control_Adh3pr,AGGGCATTTGTCGAGGGTAATACTGCATACTGCCTATTGTTCCAGA...


In [86]:
insertsite = [df_insertsite, df_insert_CG, df_insert_3mer]
df_insert_all = pd.concat(insertsite)

insert_counts = []
for index, row in df_insert_all.iterrows(): 
    sequence = row["sequence"]
    all_seq , num = mutate_sequence_motif(Seq(sequence), "CGG", "CAA")
    insert_counts.append(num)

df_insert_all['halfsite_num'] = insert_counts
df_insert_all

# cc = []
# for seq in insertsite_seq:
#     s, c = mutate_sequence_motif(Seq(seq), "CGG", "CAA")
#     cc.append(c)

,tf,type,Name,sequence,halfsite_num
0,GAL4,insert_site,GAL7pr_control,GCCAGTCCTTATCATCAACAGTTGTAGCGCTTTTTGGACCCTTTCC...,1
1,GAL4,insert_site,RUP1/SFL1pr_control,CCTTCTTCTCTTCTTTCCTCTACTACAGGTCCTTAAGCCTTGTAGT...,1
2,GAL4,insert_site,GAL2pr_control,CGAAAGGGGTTGCCTCAGGAAGGCACTCTTTCGTTGAGATATCTGC...,1
3,GAL4,insert_site,GAL1_10pr_control,CCCCATTATCTTAGCCTAAAAAAACCTTCTCTTTGGAACTTTCAGT...,1
4,YRM1,insert_site,YRM1_control_Hsv2pr,TGCACTGTTACCATAGGATCTTACCAATTTGCATTATTTCCGGCAA...,1
5,YRM1,insert_site,YRM1_GIP3/ISU1pr_control,CACTGAGACGTAATGTGGAAACACCACTTCTTACATCAAAAGGAAA...,1
6,YRM1,insert_site,YRM1_Spo24pr_control,CATGCTTTTTCATGACGCCCGGTCCTCCCCCCTGAGCCTTTGGAAG...,1
7,YRM1,insert_site,YRM1_control_Cax4pr,AGAAGCTCTGATTATTGGAAAAGGGTCTTATTCTGTACGGGGAAAC...,1
8,TEA,insert_site,TEA1_control_Isf1pr,AAGAAATTAAAAGTGTAATTGTATTTATAATAGTGAGTTTCGGAAC...,1
9,TEA,insert_site,TEA1_control_Adh3pr,AGGGCATTTGTCGAGGGTAATACTGCATACTGCCTATTGTTCCAGA...,1


In [87]:
#check_for_site(df_insert_all)

# Can half-sites increase binding at singlton or weak site?

In [88]:
df_sing = pd.read_csv('TF_singlton_sequences.csv',  header= None , sep=',')
df_sing.columns = ["tf","Name", "sequence", "motif"]


mut_motif = []
mut_CAA_seq = []
remove_site = []
motif_Len = []
T = []
N = []
motif = []
ty_name = []
ty = "WT_singlton"
for index, row in df_sing.iterrows(): 
    seq = row["sequence"]
    n = row["Name"]
    tf = row["tf"]
    m = row["motif"]
    motif_L = len(m)
    motif_Len.append(motif_L)
    #print(seq)
    mut_seqp, cc = mutate_sequence_motif(Seq(seq), m, "P"* len(m))
    ty_name.append(ty)
    #print(str(mut_seqp))
    mut_motif.append(str(mut_seqp))
    mut_CAA, c = mutate_sequence_motif(mut_seqp, "CGG", "CAA")
    #print(c)
    mut, c = mutate_sequence_motif(mut_CAA, "P"* len(m) , m)
    mut_CAA_seq.append(str(mut))
    #print(c)
    mut_remove_CGG, count = mutate_sequence_motif(mut_seqp, "CGG","")    
    #remove_site.append(str(mut_remove_CGG))
    mut_final, c1 = mutate_sequence_motif(mut_remove_CGG, "CGG","") 
    remove_site.append(str(mut_final))
    N.append(n)
    T.append(tf)
    motif.append(m)
    
    
df = pd.DataFrame(list(zip(T , N, motif_Len, remove_site, motif)), columns =['tf','Name','motif_length', 'sequence', 'motif']) 
df_sing= df_sing.drop(['motif'], axis=1)
df_sing.insert(1, 'type', ty)
df_sing

,tf,type,Name,sequence
0,﻿GAL4,WT_singlton,Rio_Gcy1pr,GGAGTGTATAAGAATTGTAAGTTATAACACCGGCGAACAATCGGGG...
1,GAL4,WT_singlton,Tim9pr,CCTTTATATATAATGTGGTTATATACCAACCTTTTGTAAATTTCCA...
2,GAL4,WT_singlton,Rup1_Sfl1pr,CCGGCTTCTTCTCTTCTTTCCTCTACTCCGACAGGTCCTTAAGCCT...
3,GAL4,WT_singlton,Rio_Gcy1pr_mut_can,GGAGTGTATAAGAATTGTAAGTTATAACACCGGCGAACAATCAAGG...
4,GAL4,WT_singlton,Tim9pr_mut_can,CCTTTATATATAATGTGGTTATATACCAACCTTTTGTAAATTTCCA...
5,GAL4,WT_singlton,Rup1_Sfl1pr_mut_can,CCGGCTTCTTCTCTTCTTTCCTCTACTCCGACAGGTCCTTAAGCCT...
6,LEU3,WT_singlton,Vph1pr,CCAATTACCTCAATCTGGTACGCTCCGGTACCGGACGCCAGTACGA...
7,LEU3,WT_singlton,Bap2pr,TAGCAGCATTTGCTGCTGACATATTTACCGTTGAAGGCCATCATAA...
8,LEU3,WT_singlton,Leu4pr,AAGTATGACACACATTCAACTCAGCTTTTCTGAGGCCTTCTTTCTT...
9,LEU3,WT_singlton,Vph1pr_mut_can,CCAATTACCTCAATCTGGTACGCTCCAATACCAAACGCCAGTACGA...


In [89]:
new_df = df[~df['tf'].str.contains("YRM1")]
YRM1_df = df[df['tf'].str.contains("YRM1")]
#yrm1_df = YRM1_df.drop(['motif', 'motif_length'], axis = 1)

yrm1_singlton_seq = []
yrm1_singlton_tf = []
yrm1_singlton_type = []
yrm1_singlton_name = []
yrm1_t = "singlton"

for index, row in YRM1_df.iterrows():
    seq = row['sequence']
    TF = row['tf']
    n = row['Name']
    m = row['motif']
    ml =['motif_length']
    #b, p = mutate_sequence_motif(Seq(seq), "CGG", "CAA")
    s, c = mutate_sequence_motif(Seq(seq),"P" * len(m), m)
    #print(p)
    
    yrm1_singlton_seq.append(str(s))
    yrm1_singlton_tf.append(tf)
    yrm1_singlton_type.append(yrm1_t)
    yrm1_singlton_name.append(n)
    
    
# #yrm1_df.pop('motif_Len')
df_yrm1 = pd.DataFrame(list(zip(yrm1_singlton_tf ,yrm1_singlton_type, yrm1_singlton_name , yrm1_singlton_seq)), columns =['tf','type','Name','sequence']) 

df_yrm1

,tf,type,Name,sequence
0,YRM1,singlton,Nrt1_Sgo1pr,GCCAATGTTCGCATCTATGAGGGAGATGAACTCGCTCTTTGGGTTT...
1,YRM1,singlton,Gip3_Isu1pr,GCTGCCATGATGATTGCATCACTGAGACGTAATGTGGAAACACCAC...
2,YRM1,singlton,Spo24pr,CGCCTCCTCCCCGCCCCTGAGCCTTTGGAAGTGTATTAATGGGCGC...
3,YRM1,singlton,Nrt1_Sgo1pr_mut_canon,GCCAATGTTCGCATCTATGAGGGAGATGAACTCGCTCTTTGGGTTT...
4,YRM1,singlton,Gip3_Isu1pr_mut_canon,GCTGCCATGATGATTGCATCACTGAGACGTAATGTGGAAACACCAC...
5,YRM1,singlton,Spo24pr_mut_canon,CGCCTCCTCCAAGCCCCTGAGCCTTTGGAAGTGTATTAATGGGCGC...


In [90]:
# seq ='GGAGTGTATAAGAATTGTAAGTTATAACAAACAATPPPPPPPPPPPPPPPPPGGGAAGAACAAGGAAGGGTCTTTTCTCCCTCATTGTCATAGCAAGGTCATTTCGCCTTCTCAGAAAGGGGTAGAATCAATCTAGCACGCAGATTGCAAACACTTAA'
# len(seq)

In [91]:
#this works for adding half-sites to pomoters with singlton/weak site 

def singlton(sequence, halfsite_num, motif_length):
    s = sequence 
    len_seq = len(sequence)
    indices_start = [m.start() for m in re.finditer("P", sequence)]
    indices_end = [m.end() for m in re.finditer("P", sequence)]
    indicesList= list(set(indices_start) | set(indices_end))
    #print(indicesList)
    
    number_insert = 3 * halfsite_num
    #print(number_insert)
    #print(number_insert)
    if len_seq + number_insert > 170:
        #print(len_seq)
        numb = (len_seq + number_insert) - 170
        #print(numb)
        #print(len_seq)
        n = len_seq-numb
        #print(n)
        final_seq = s[:n]
        #print(len(final_seq))
        last_i = len(final_seq)-1
        
        
        
        #in indicesList or 
        list_numbers = []
        for number in range(1, last_i, 3):
            if number in indicesList or number == 0 or number == 1:
                continue 
            else: 
                list_numbers.append(number)
        #print(list_numbers)
        random_indices = random.sample(list_numbers, (halfsite_num)) 
        #print(len(random_indices))
#         for index in random_indices:
#             if index == 0 or index == 1:
#                 random_indices.remove(index)
        
#         print(len(random_indices))
        
     

        first_index = [0]
        last_index = [len(final_seq)+1]
        #lists are combined and sorted smallest to largest
        random_indices.extend(last_index)
        random_indices.extend(first_index)
        
        
# #        
        
        random_indices.sort()
        
        
        final_index_list = []
      
#         # #uses the indices to split the sequence into "words"
        [final_index_list.append(x) for x in random_indices if x not in final_index_list] 
        #print(final_index_list)
        #print(final_index_list)
        parts = [final_seq[i:j] for i,j in zip(final_index_list, final_index_list[1:])]
        #creates indices where there isnt a half-site 
        site = [i for i, e in enumerate(parts) if e != "CGG"]
        site.remove(0)
        #will add a half-site to indices without a site and returns a new sequence 
        lista = ['CGG'] * len(site)
        assert(len(lista)) == len(site)
        acc = 0
        for i in range(len(lista)):
            parts.insert(site[i]+acc, lista[i])
            acc += 1
        
        new_seqs = "".join(parts)
        
        
#         for s in new_seqs:
        
#             motif = check_for_site(s, TF)

#             if motif == 1:
#                 break
#             else: 
#                 return new_seqs
        
        

        b = [sequence[i:i+ motif_length] for i in xrange(len(new_seqs))]
        for seq in b:
            s, c = mutate_sequence_motif(Seq(seq), "CGG", "XYZ")
            new_seq = str(s)
            indices = [m.start() for m in re.finditer("XYZ", new_seq)]
            if 0 in indices and motif_length in indices:
                break
            else: 
                return "".join(parts)
            
            
        
        return "".join(parts)

    
    



In [92]:
#print(remove_site)
#is going to add 16 half-sites to each singlton promoter while making sure to not create a full site 

singlton_seq =[]
singlton_tf = []
singlton_pr_name = []
singlton_type_mut = []
t = "singlton"



for index, row in new_df.iterrows(): 
    ml = row["motif_length"]
    seq = row["sequence"]
    tf = row["tf"]
    #print(tf)
    n =row["Name"]
    m = row['motif']
    
    #Eml = len(m)
  
    new_half = singlton(seq, 16, ml)

    #print(new_half)
    s, c = mutate_sequence_motif(Seq(new_half),"P" * len(m), m)
    singlton_seq.append(str(s))
    singlton_tf.append(tf)
    singlton_pr_name.append(n)
    singlton_type_mut.append(t)

df_singlton = pd.DataFrame(list(zip(singlton_tf , singlton_type_mut, singlton_pr_name , singlton_seq)), columns =['tf', 'type', 'Name','sequence']) 

df_singlton

,tf,type,Name,sequence
0,﻿GAL4,singlton,Rio_Gcy1pr,GGAGCGGTGTCGGATACGGAGAATTGTACGGAGTCGGTATAACAAA...
1,GAL4,singlton,Tim9pr,CCTTTATATACGGTAACGGTGTGGTTATCGGATACCACGGACCTTT...
2,GAL4,singlton,Rup1_Sfl1pr,CCTTCTTCTCTTCCGGTTTCGGCCTCGGCTACGGCTACAGGTCCTT...
3,GAL4,singlton,Rio_Gcy1pr_mut_can,GGAGCGGTGTCGGATACGGAGAATTGTACGGAGTCGGTATCGGAAC...
4,GAL4,singlton,Tim9pr_mut_can,CCTTTATCGGATACGGTAACGGTGTCGGGGTTATCGGATACCACGG...
5,GAL4,singlton,Rup1_Sfl1pr_mut_can,CCTTCTTCTCCGGTTCCGGTTTCGGCCTCTACGGCTACGGCAGGTC...
6,LEU3,singlton,Vph1pr,CCAATTACGGCCTCAACGGTCTCGGGGTACGCTCCGGTACCGGACG...
7,LEU3,singlton,Bap2pr,TAGCCGGAGCATTCGGTGCCGGTGCTGACATATTTATCGGTGACGG...
8,LEU3,singlton,Leu4pr,AAGTATGACACACCGGATTCAACTCAGCCGGTTTCGGTCTGAGGCC...
9,LEU3,singlton,Vph1pr_mut_can,CCAACGGTTACCTCAACGGTCTGGTACGCTCCAATACCAAACGCGG...


In [93]:
#works as CG control for singlton 

def singlton_CG_control(sequence, halfsite_num):
    
    s = sequence 
    len_seq = len(sequence)
    indices_start = [m.start() for m in re.finditer("P", sequence)]
    indices_end = [m.end() for m in re.finditer("P", sequence)]
    indicesList= list(set(indices_start) | set(indices_end))
    
    
    CG_bases = halfsite_num * 3
    num_bases_add = int((CG_bases/2))

    ss_list = list(sequence)
    list_int1 = []
    #list_int2 = []
    for i in range(len(ss_list)):
        if ss_list[i] == "A" or ss_list[i] == "T":
            list_int1.append(i)
    
    list_int1 = list_int1[:55]
    if len_seq + CG_bases > 170:
        numb = (len_seq + CG_bases) - 170
        n = len_seq-numb
        final_seq = seq[:n]
        last_i = len(final_seq)-1
        
        
        for num in list_int1:
            if num in indicesList or num == 0 or num == len_seq:
                list_int1.remove(num)
                
        
        list_ind = random.sample(list_int1, num_bases_add) 

        

        first_index = [0]
        last_index = [len(seq)+1]
        #lists are combined and sorted smallest to largest
        list_ind.extend(last_index)
        list_ind.extend(first_index)
        list_ind.sort()
        
        #print(list_ind)


        final_index_list = []
        


        # #uses the indices to split the sequence into "words"
        [final_index_list.append(x) for x in list_ind if x not in final_index_list] 
        #print(final_index_list)
        parts = [final_seq[i:j] for i,j in zip(final_index_list, final_index_list[1:])]
        #creates indices where there isnt a half-site 
        site = [i for i, e in enumerate(parts) ]
        site.remove(0)
        

        # will add a half-site to indices without a site and returns a new sequence 
        lista = ['CC'] * len(site)
        assert(len(lista)) == len(site)
        acc = 0
        for i in range(len(lista)):
            parts.insert(site[i]+acc, lista[i])
            acc += 1
        new_seq = "".join(parts)
        
        
#         for s in new_seq:
        
#             motif = check_for_site(s, TF)

#             if motif == 1:
#                 break
#             else: 
#                 return new_seq
        
        
        
        return new_seq

    

In [94]:
def singlton_CG_control2(sequence):
    
    s = sequence 
    len_seq = len(sequence)
    indices_start = [m.start() for m in re.finditer("P", sequence)]
    indices_end = [m.end() for m in re.finditer("P", sequence)]
    indicesList= list(set(indices_start) | set(indices_end))
    
    
    CG_bases = 170 - len(sequence) 
    #print(CG_bases)
    
    #halfsite_num * 3
    num_bases_add = int((CG_bases/2))
    #print(num_bases_add)

    ss_list = list(sequence)
    list_int1 = []
    #list_int2 = []
    for i in range(len(ss_list)):
        if ss_list[i] == "A" or ss_list[i] == "T":
            list_int1.append(i)
    
    list_int1 = list_int1[:55]
#     if len_seq + CG_bases > 170:
#         numb = (len_seq + CG_bases) - 170
#         n = len_seq-numb
#         final_seq = seq[:n]
    last_i = len(sequence)-1


    for num in list_int1:
        if num in indicesList or num == 0 or num == len_seq:
            list_int1.remove(num)


    list_ind = random.sample(list_int1, num_bases_add) 



    first_index = [0]
    last_index = [len(seq)+1]
    #lists are combined and sorted smallest to largest
    list_ind.extend(last_index)
    list_ind.extend(first_index)
    list_ind.sort()

    #print(list_ind)


    final_index_list = []



    # #uses the indices to split the sequence into "words"
    [final_index_list.append(x) for x in list_ind if x not in final_index_list] 
    #print(final_index_list)
    parts = [sequence[i:j] for i,j in zip(final_index_list, final_index_list[1:])]
    #creates indices where there isnt a half-site 
    site = [i for i, e in enumerate(parts) ]
    site.remove(0)


    # will add a half-site to indices without a site and returns a new sequence 
    lista = ['CC'] * len(site)
    assert(len(lista)) == len(site)
    acc = 0
    for i in range(len(lista)):
        parts.insert(site[i]+acc, lista[i])
        acc += 1
    new_seq = "".join(parts)
    
#     for s in new_seq:

#         motif = check_for_site(s, TF)

#         if motif == 1:
#             break
#         else: 
#             return new_seq
    
    return new_seq

#     else:
        
    

In [95]:
# s = singlton_CG_control(seq, 16)
# mutate_sequence_motif(Seq(s), "CGG", "CAA")
singlton_CG_control_seq = []
singlton_CG_control_TF = []
singlton_CG_control_pr_name = []
singlton_CG_mut_type = []
t2 = "singlton_CG_control"

for index, row in new_df.iterrows(): 
    ml = row["motif_length"]
    seq = row["sequence"]
    tf = row["tf"]
    n =row["Name"]
    m = row['motif']
  
    control_singlton = singlton_CG_control(seq, 16)
    final_s, c=mutate_sequence_motif(Seq(control_singlton), "P" * len(m) , m)
    singlton_CG_control_seq.append(str(final_s))
    singlton_CG_control_TF.append(tf)
    singlton_CG_control_pr_name.append(n)
    singlton_CG_mut_type.append(t2)
    #print(final_s)
    
    #print(new_half)
    #s, c = mutate_sequence_motif(Seq(new_half),"P" * len(m), m)
df_singlton_CG_control = pd.DataFrame(list(zip(singlton_CG_control_TF ,singlton_CG_mut_type, singlton_CG_control_pr_name , singlton_CG_control_seq)), columns =['tf','type','Name','sequence']) 

    


In [96]:
yrm1_singlton_control_seq = []
yrm1_singlton_control_tf = []
yrm1_singlton_control_pr_name = []
yrm1_mut_type = []


for index, row in YRM1_df.iterrows(): 
    ml = row["motif_length"]
    seq = row["sequence"]
    tf = row["tf"]
    n =row["Name"]
    m = row['motif']
    control_seq = singlton_CG_control2(seq)
    s ,c= mutate_sequence_motif(Seq(control_seq), "P" * len(m) , m)
    yrm1_singlton_control_seq.append(str(s))
    yrm1_singlton_control_tf.append(tf)
    yrm1_singlton_control_pr_name.append(n)
    yrm1_mut_type.append(t2)
    #print(s)
    
df_yrm1_singlton_control = pd.DataFrame(list(zip(yrm1_singlton_control_tf , yrm1_mut_type, yrm1_singlton_control_pr_name , yrm1_singlton_control_seq)), columns =['tf','type','Name','sequence']) 


In [97]:
singlton_count = []
final_singlton_seq = [df_sing ,df_singlton, df_yrm1, df_singlton_CG_control, df_yrm1_singlton_control]
df_singlton_seq = pd.concat(final_singlton_seq)
for index, row in df_singlton_seq.iterrows(): 
    seq = row['sequence']
    s,c= mutate_sequence_motif(Seq(str(seq)), 'CGG', "CAA")
    singlton_count.append(c)
    
df_singlton_seq['halfsite_num'] = singlton_count
df_singlton_seq

,tf,type,Name,sequence,halfsite_num
0,﻿GAL4,WT_singlton,Rio_Gcy1pr,GGAGTGTATAAGAATTGTAAGTTATAACACCGGCGAACAATCGGGG...,5
1,GAL4,WT_singlton,Tim9pr,CCTTTATATATAATGTGGTTATATACCAACCTTTTGTAAATTTCCA...,3
2,GAL4,WT_singlton,Rup1_Sfl1pr,CCGGCTTCTTCTCTTCTTTCCTCTACTCCGACAGGTCCTTAAGCCT...,9
3,GAL4,WT_singlton,Rio_Gcy1pr_mut_can,GGAGTGTATAAGAATTGTAAGTTATAACACCGGCGAACAATCAAGG...,3
4,GAL4,WT_singlton,Tim9pr_mut_can,CCTTTATATATAATGTGGTTATATACCAACCTTTTGTAAATTTCCA...,1
5,GAL4,WT_singlton,Rup1_Sfl1pr_mut_can,CCGGCTTCTTCTCTTCTTTCCTCTACTCCGACAGGTCCTTAAGCCT...,7
6,LEU3,WT_singlton,Vph1pr,CCAATTACCTCAATCTGGTACGCTCCGGTACCGGACGCCAGTACGA...,6
7,LEU3,WT_singlton,Bap2pr,TAGCAGCATTTGCTGCTGACATATTTACCGTTGAAGGCCATCATAA...,7
8,LEU3,WT_singlton,Leu4pr,AAGTATGACACACATTCAACTCAGCTTTTCTGAGGCCTTCTTTCTT...,6
9,LEU3,WT_singlton,Vph1pr_mut_can,CCAATTACCTCAATCTGGTACGCTCCAATACCAAACGCCAGTACGA...,4


# Making the spacing mutation
I will be making 5 oligos per promoter sequence. I will be adding 2, 4, 6, 8, 10 bp between each CGG half-site. Because the sequence length will increase, I removed bp at the end of the sequence to make sure the oligo length stays 170 bp. Because I will be removing bp I will lose half-sites the larger the spacing. This could also effect expression but I will have mut sequences I can normalize it to.   

https://stackoverflow.com/questions/19845745/python-move-characters-based-on-string-position/19858225

In [98]:
# sequence = "CGGATTCGGTACGGATCAACGGTTCGGCGG"
# halfsite = "CGG"


#this works to space out each half-site by 2 
def increment(orig_seq, halfsite, space, TF):
    seq, count = mutate_sequence_motif(Seq(sequence), "CGG", "")
    s = str(seq)
    last_i = len(s)


    index_list = []
    for index in range(0,last_i,space):
        index_list.append(index)
        
    char_list = list(s)


    list_b = ["CGG"] * len(index_list)
    assert(len(list_b) == len(index_list))
    acc = 0
    for i in range(len(list_b)):
        char_list.insert(index_list[i]+acc, list_b[i])
        acc += 1
    
    new_seq = "".join(char_list)
    
    
    for s in new_seq:

        motif = check_for_site(s, TF)

        if motif == 1:
            break
        else: 
            return new_seq

    
#     ns = [new_seq[i:i+17] for i in xrange(len(new_seq))]
#     for seq in ns:
#         s, c = mutate_sequence_motif(Seq(seq), "CGG", "XYZ")
#         ns_string = str(s)
#         indices = [m.start() for m in re.finditer("XYZ", ns_string)]
#         if 0 in indices and 14 in indices:
#             break
#         else: 
#             return "".join(new_seq)            



    

    #return "".join(char_list)


In [99]:
#  

In [100]:
#increment(sequence,halfsite, 9)

In [101]:
#I THINK THIS FINALLY WORKS 

def blah(sequence,space_len, TF):





    last_i = len(sequence)
    
    # print(list_nonCGG)
    seq, count = mutate_sequence_motif(Seq(sequence),"CGG","P")
    new_seq = str(seq)
    indices_end = [m.start() for m in re.finditer("P", new_seq)]
    seq2, count = mutate_sequence_motif(Seq(new_seq),"P","")
    final = str(seq2)
    seq3, count = mutate_sequence_motif(Seq(final),"CGG","CAA")
    finalseq = str(seq3)
    


    l = len(indices_end) * space_len
    increment_list = []
    for increment in range(0,l,space_len):
        increment_list.append(increment)


    res_list = [] 
    for i in range(0, len(indices_end)): 
        new = indices_end[i] + increment_list[i]
        #print(new)
        if new <= len(finalseq):
            res_list.append(new)
            





    char_list = list(finalseq)


    list_b = ["CGG"] * len(res_list)
    # #     print("list_b")
    # #     print(list_b)
    assert(len(list_b) == len(res_list))
    acc = 0
    # #     print(">final res")
    # #     print(res)
    for i in range(len(list_b)):
        char_list.insert(res_list[i]+acc, list_b[i])
        acc+= 0
        
    new_seq ="".join(char_list)
    
    
    for s in new_seq:

        motif = check_for_site(s, TF)

        if motif == 1:
            break
        else: 
            return new_seq



#     ns = [new_seq[i:i+17] for i in xrange(len(new_seq))]
#     for seq in ns:
#         s, c = mutate_sequence_motif(Seq(seq), "CGG", "XYZ")
#         ns_string = str(s)
#         indices = [m.start() for m in re.finditer("XYZ", ns_string)]
#         if 0 in indices and 14 in indices:
#             break
#         else: 
#             return "".join(new_seq)            


        
        

    #return "".join(char_list)

In [102]:
df = pd.read_csv('Lib_control_seq_final.csv',  header= None , sep=',')
df.columns = ["tf","name", "sequence"]
#df_seq = df[~df['name'].str.contains("control")]
#df_seq = df[~df['name'].str.contains("RGT")]
#df_seq = df_seq.loc[df_seq['halfsite_num'] > 1]




df

,tf,name,sequence
0,﻿GAL4,ADR1pr_reg1,GTACGGCATCCGCGCGGCTCCGATCGCCCGTCCTACTTTTGCGGCC...
1,GAL4,HAP4pr_reg1,CCCGGGCCCCGGAGGTTGATCGCACGGCAAATACATAAGAACCTTC...
2,GAL4,STP4pr_reg1,CCTCCGGCTGCTTTTTAGCCTTCTACGCGCGGGACGGCTTGCGCGC...
3,GAL4,GID8pr_reg1,TCGGCTTGGGTTTGTGTGTACGGCCCTTTTTGGGATCCCGGAGTTT...
4,GAL4,GAL1_10pr,CCCCATTATCTTAGCCTAAAAAAACCTTCTCTTTGGAACTTTCAGT...
5,YRM1,YRM1_Mpr20pr,CCGCGGGATGCCATGCACGGATGCCTGCGGAGACAGTGGCGCGGAG...
6,YRM1,YRM1_Mnn4pr,GTGCCGTAGTCTGTATACGGTTTTAGATGCGGTATGTTTATCATCG...
7,YRM1,YRM1_Tpo4/Mod5pr,CGGAGCCCCTGCAGCTTCATGAAGACGGAACCGCACGGGCCTATTA...
8,YRM1,YRM1_control_Hsv2pr,TGCACTGTTACCATAGGATCTTACCAATTTGCATTATTTCCGCCAA...
9,YRM1,YRM1_control_Cax4pr,AGAAGCTCCGCTGATTATCCGTGGAAAAGGGTCTTATTCTGCCGTA...


In [103]:
space = "space_mutation"
increment_orientation = 'space_orientation'


space_mutname = []
space_seq = []
mut_type = []
tf_n = []

increment_orientation_seq = []
increment_type = []
increment_tf = []
increment_name = []



for i in range(2,11,2):
    for index, row in df.iterrows(): 
        s = row["sequence"]
        n = row["name"]
        t = row["tf"]
        #print(t)
        space_mut = blah(s,i,t)
        increment_mut, hs, num = (s,"CGG", i)
        #print(increment_mut)
        final_increment = space_orientation(increment_mut)
        
        
        space_seq.append(space_mut)
        name = n 
        space_mutname.append(name)
        mut_type.append(space)
        tf_n.append(t)
        
        increment_orientation_seq.append(final_increment)
        increment_type.append(increment_orientation)
        increment_tf.append(t)
        increment_name.append(n)
        
increment_df =  pd.DataFrame(list(zip(increment_tf , increment_type, increment_name, increment_orientation_seq)), 
               columns =['tf','type','Name', 'sequence'])  
increment_df

<analyze_sig_hits_v2.pssm instance at 0x2ae0d0b13c68>
0
<analyze_sig_hits_v2.pssm instance at 0x2ae0d3df1f38>
0
<analyze_sig_hits_v2.pssm instance at 0x2ae0d3df1200>
0
<analyze_sig_hits_v2.pssm instance at 0x2ae0d3df1560>
0
<analyze_sig_hits_v2.pssm instance at 0x2ae0d3df1b00>
0
<analyze_sig_hits_v2.pssm instance at 0x2ae0d3df1bd8>
0
<analyze_sig_hits_v2.pssm instance at 0x2ae0d7887290>
0
<analyze_sig_hits_v2.pssm instance at 0x2ae0d3df1320>
0
<analyze_sig_hits_v2.pssm instance at 0x2ae0d77d23f8>
0
<analyze_sig_hits_v2.pssm instance at 0x2ae0d77d2b00>
0
<analyze_sig_hits_v2.pssm instance at 0x2ae0d77d27a0>
0
<analyze_sig_hits_v2.pssm instance at 0x2ae0d77d2758>
0
<analyze_sig_hits_v2.pssm instance at 0x2ae0d77d2098>
0
<analyze_sig_hits_v2.pssm instance at 0x2ae0d77d2878>
0
<analyze_sig_hits_v2.pssm instance at 0x2ae0d77d2c20>
0
<analyze_sig_hits_v2.pssm instance at 0x2ae0d77d2680>
0
<analyze_sig_hits_v2.pssm instance at 0x2ae0d77d26c8>
0
<analyze_sig_hits_v2.pssm instance at 0x2ae0d77d

,tf,type,Name,sequence
0,﻿GAL4,space_orientation,ADR1pr_reg1,GTACCGCATCGGCGCGGCTCCGATCGCCGGTCCTACTTTTGCCGCT...
1,GAL4,space_orientation,HAP4pr_reg1,TTCGGGCCCCCGAGGTTGATCGCACGGCAAATACATAAGAACCTTC...
2,GAL4,space_orientation,STP4pr_reg1,CCTCCGGCTGCTTTTTAGCCTTCTACGCACGGAACCGCTTGCGCGC...
3,GAL4,space_orientation,GID8pr_reg1,TCCGCTTGGGTTTGTGTGTACCGCCCTTTTTGGGATCCCCGAGTTT...
4,GAL4,space_orientation,GAL1_10pr,CCCCATTATCTTAGCCTAAAAAAACCTTCTCTTTGGAACTTTCAGT...
5,YRM1,space_orientation,YRM1_Mpr20pr,CGGCGGGATGCCATGCACCGATGCCTGCCGAGACAGTGGCGCGGAG...
6,YRM1,space_orientation,YRM1_Mnn4pr,GTGCGGTAGTCTGTATACCGTTTTAGATGCGGTATGTTTATCATCG...
7,YRM1,space_orientation,YRM1_Tpo4/Mod5pr,CCGAGCCCCTGCAGCTTCATGAAGACGGAACGGCACCGGCCTATTA...
8,YRM1,space_orientation,YRM1_control_Hsv2pr,TGCACTGTTACCATAGGATCTTACCAATTTGCATTATTTCGGCCAA...
9,YRM1,space_orientation,YRM1_control_Cax4pr,AGAAGCTCCGCTGATTATCGGTGGAAAAGGGTCTTATTCTGCCGTA...


In [104]:
spacemut_df = pd.DataFrame(list(zip(tf_n ,mut_type, space_mutname, space_seq)), 
               columns =['tf','type','Name', 'sequence']) 
spacemut_df

,tf,type,Name,sequence
0,﻿GAL4,space_mutation,ADR1pr_reg1,GTACGGCATCGCGGCTATCGGCGCTCGGCCTACTTCGGTTGCTTGA...
1,GAL4,space_mutation,HAP4pr_reg1,TTCGGGCCCAGCGGGTTGATCGCACAAACGGTACATAAGAACCTTC...
2,GAL4,space_mutation,STP4pr_reg1,CCTCCGGCTGCTTTTTAGCCTTCTACGCAAACGGCTTGCGGCGCGC...
3,GAL4,space_mutation,GID8pr_reg1,TCGGCTTGGGTTTGTGTGTACCCGGCTTTTTGGGATCCAGTTCGGT...
4,GAL4,space_mutation,GAL1_10pr,CCCCATTATCTTAGCCTAAAAAAACCTTCTCTTTGGAACTTTCAGT...
5,YRM1,space_mutation,YRM1_Mpr20pr,CGGGACGGTGCCATGCAATGCCGGCTGAGACAGCGGTGGCGAGTCA...
6,YRM1,space_mutation,YRM1_Mnn4pr,GTGCGGTAGTCTGTATATTCGGTTAGATGTATGCGGTTTATCATCG...
7,YRM1,space_mutation,YRM1_Tpo4/Mod5pr,CGGAGCCCCTGCAGCTTCATGAAGAAACGGCAGCCGGCTATCGGTA...
8,YRM1,space_mutation,YRM1_control_Hsv2pr,TGCACTGTTACCATAGGATCTTACCAATTTGCATTATTTCGGCCAA...
9,YRM1,space_mutation,YRM1_control_Cax4pr,AGAAGCTCGGCTGATTATTGCGGGAAAAGGGTCTTATTCTGTAGGC...


In [105]:
space_all = [increment_df, spacemut_df]
df_space_final =pd.concat(space_all)

In [106]:
counts = []
for index, row in df_space_final.iterrows():
    seq = row['sequence'] 
    s, c = mutate_sequence_motif(Seq(seq),"CGG","CAA")
    counts.append(c)

In [107]:
df_space_final["halfsite_num"] = counts

In [108]:
#spacemut_df.to_csv("FINAL_spacemut.csv") 

In [109]:
#check_for_site(spacemut_df)

In [110]:
# s, count = mutate_sequence_motif(Seq("GTACGGCATCCGCGCGGCTCCGATCGCCCGTCCTACTTTTGCGGCCCGGCGCCGACGGCCTCCCCGGAATTTACCAACGAAGAGGAAAATAAACTGGAGTTATAACGCCATTGGCGGGAAACTCCGAAACCATAGCGCCGTGGTCTTTCCCCGGTTTTTTGGCGCCGGGT"), "CGG", "PPP")

# Make same mutant sequences for 4 zinc clusters¶
Will be making the same mutations that Rob made for the Gal4 half-site library. 

|Name|
|----|
|TFpr_reg1  |
|TFpr_reg1_mut_can  |
|TFpr_reg1 _mutCGG1|
|TFpr_reg1 _mutCGG2|
|TFpr_reg1 _ranmut1|
|TFpr_reg1 _ranmut2|

In [149]:
df_WT = pd.read_csv('Lib_control_seq_final.csv',  header= None , sep=',')
df_mutcan = pd.read_csv('Lib_controls_mutcan_final.csv',  header= None , sep=',')
df_WT.columns = ["tf","Name", "sequence"]
df_mutcan.columns = ["tf","type","Name", "sequence"]
# #df = df[~df['name'].str.contains("RGT")]


In [150]:
wt = ['WT'] * 24
df_WT.insert(2, "type", wt)


In [151]:
v1 = "mutCGG1"
v2 = "mutCGG2"
v3 = "ranmut1"
v4 = "ranmut2"


Names = []
Sequences = []
mutationt = []
tf_n = []

for index, row in df.iterrows(): 
    seq = Seq(row["sequence"])
    n = row["name"]
    tf = row["tf"]
    mutCGG1, num = mutate_sequence_motif(seq ,Seq("CGG"),Seq("CAA"))
    Sequences.append(str(mutCGG1))
    name_mutCGG1 = n
    Names.append(name_mutCGG1)
    mutationt.append(v1)
    tf_n.append(tf)
    mutCGG2, num = mutate_sequence_motif(seq ,Seq("CGG"),Seq("CTT"))
    Sequences.append(str(mutCGG2))
    name_mutCGG2 = n
    Names.append(name_mutCGG2)
    mutationt.append(v2)
    tf_n.append(tf)
    ranmut1 = mutate_sequence_random(seq,Seq("CGG"),Seq("AA"),num)
    Sequences.append(str(ranmut1))
    name_ranmut1 = n
    tf_n.append(tf)
    Names.append(name_ranmut1)
    mutationt.append(v3)
    ranmut2 = mutate_sequence_random(seq,Seq("CGG"),Seq("TT"),num)
    Sequences.append(str(ranmut2))
    name_ranmut2 = n
    tf_n.append(tf)
    Names.append(name_ranmut2)
    mutationt.append(v4)
    

In [152]:
typesmuts_df = pd.DataFrame(list(zip(tf_n,mutationt,Names, Sequences)), 
               columns =["tf",'type','Name', 'sequence']) 
#final_mut_types = [df_WT, typesmuts_df]

In [153]:
combined_mut_df = [df_WT ,df_mutcan, typesmuts_df]

In [154]:
final_mut_types= pd.concat(combined_mut_df)

In [155]:
halfsite_n = []
for index, row in final_mut_types.iterrows(): 
    seq = Seq(row["sequence"])
    #n = row["name"]
    mutCGG1, num = mutate_sequence_motif(seq ,Seq("CGG"),Seq("CAA"))
    halfsite_n.append(num)

final_mut_types['halfsite_num']= halfsite_n 

In [156]:
final_mut_types

,Name,sequence,tf,type,halfsite_num
0,ADR1pr_reg1,GTACGGCATCCGCGCGGCTCCGATCGCCCGTCCTACTTTTGCGGCC...,﻿GAL4,WT,15
1,HAP4pr_reg1,CCCGGGCCCCGGAGGTTGATCGCACGGCAAATACATAAGAACCTTC...,GAL4,WT,11
2,STP4pr_reg1,CCTCCGGCTGCTTTTTAGCCTTCTACGCGCGGGACGGCTTGCGCGC...,GAL4,WT,10
3,GID8pr_reg1,TCGGCTTGGGTTTGTGTGTACGGCCCTTTTTGGGATCCCGGAGTTT...,GAL4,WT,9
4,GAL1_10pr,CCCCATTATCTTAGCCTAAAAAAACCTTCTCTTTGGAACTTTCAGT...,GAL4,WT,8
5,YRM1_Mpr20pr,CCGCGGGATGCCATGCACGGATGCCTGCGGAGACAGTGGCGCGGAG...,YRM1,WT,14
6,YRM1_Mnn4pr,GTGCCGTAGTCTGTATACGGTTTTAGATGCGGTATGTTTATCATCG...,YRM1,WT,11
7,YRM1_Tpo4/Mod5pr,CGGAGCCCCTGCAGCTTCATGAAGACGGAACCGCACGGGCCTATTA...,YRM1,WT,14
8,YRM1_control_Hsv2pr,TGCACTGTTACCATAGGATCTTACCAATTTGCATTATTTCCGCCAA...,YRM1,WT,5
9,YRM1_control_Cax4pr,AGAAGCTCCGCTGATTATCCGTGGAAAAGGGTCTTATTCTGCCGTA...,YRM1,WT,12


# Combine all dataframes 
This will generate the complete list of promoter sequences for my library

In [157]:
CG_control = pd.read_csv('CG_control.csv',  header= None , sep=',')
CG_control.columns = ["tf",'type',"Name", "sequence",'halfsite_num']



In [158]:
# final_lib_seq = [final_sampled_mut, spacemut_df, df_orientation_fin, df_insertsite, final_mut_types]
final_lib_seq = [final_mut_types, final_sampled_mut, fr, CG_control, df_orientation_fin, df_insert_all, df_singlton_seq, df_space_final]

In [159]:
FINAL_promoter_lib = pd.concat(final_lib_seq)
FINAL_promoter_lib

,Name,halfsite_num,sequence,tf,type
0,ADR1pr_reg1,15,GTACGGCATCCGCGCGGCTCCGATCGCCCGTCCTACTTTTGCGGCC...,﻿GAL4,WT
1,HAP4pr_reg1,11,CCCGGGCCCCGGAGGTTGATCGCACGGCAAATACATAAGAACCTTC...,GAL4,WT
2,STP4pr_reg1,10,CCTCCGGCTGCTTTTTAGCCTTCTACGCGCGGGACGGCTTGCGCGC...,GAL4,WT
3,GID8pr_reg1,9,TCGGCTTGGGTTTGTGTGTACGGCCCTTTTTGGGATCCCGGAGTTT...,GAL4,WT
4,GAL1_10pr,8,CCCCATTATCTTAGCCTAAAAAAACCTTCTCTTTGGAACTTTCAGT...,GAL4,WT
5,YRM1_Mpr20pr,14,CCGCGGGATGCCATGCACGGATGCCTGCGGAGACAGTGGCGCGGAG...,YRM1,WT
6,YRM1_Mnn4pr,11,GTGCCGTAGTCTGTATACGGTTTTAGATGCGGTATGTTTATCATCG...,YRM1,WT
7,YRM1_Tpo4/Mod5pr,14,CGGAGCCCCTGCAGCTTCATGAAGACGGAACCGCACGGGCCTATTA...,YRM1,WT
8,YRM1_control_Hsv2pr,5,TGCACTGTTACCATAGGATCTTACCAATTTGCATTATTTCCGCCAA...,YRM1,WT
9,YRM1_control_Cax4pr,12,AGAAGCTCCGCTGATTATCCGTGGAAAAGGGTCTTATTCTGCCGTA...,YRM1,WT


In [160]:
FINAL_promoter_lib.to_csv("FINAL_PROMOTER_LIB.csv") 

In [122]:
def check_for_site(df):
    numb = []
    for index, row in df.iterrows():
        
        
        tf = row["tf"]
        seq = row['sequence']
        t = row['type']
        num = row['halfsite_num']
        tf = tf.replace("TF","")
        n = row["Name"]
        
        #print(tf)
        
        tf = tf.replace('\xef',"")
        tf =tf.replace('\xbb',"")
        tf =tf.replace('\xbf',"")
        m = get_max_pwm_score(seq, tf)
        #print(m)
        #motif = one_strong_or_two_weak(s,tf,m,dif=2)
        motif = one_weak_one_strong(s, tf, m, dif=3)
        print(n ,motif)
#         numb.append(motif)
#     return numb
                

In [123]:
check_for_site(df_space_final)


<analyze_sig_hits_v2.pssm instance at 0x2ae0d7882c68>
('ADR1pr_reg1\xc2\xa0', 0)
<analyze_sig_hits_v2.pssm instance at 0x2ae0d7882f38>
('HAP4pr_reg1', 0)
<analyze_sig_hits_v2.pssm instance at 0x2ae0d78826c8>
('STP4pr_reg1', 0)
<analyze_sig_hits_v2.pssm instance at 0x2ae0d7882170>
('GID8pr_reg1', 0)
<analyze_sig_hits_v2.pssm instance at 0x2ae0d7882c68>
('GAL1_10pr', 0)
<analyze_sig_hits_v2.pssm instance at 0x2ae0d7882f38>
('YRM1_Mpr20pr', 0)
<analyze_sig_hits_v2.pssm instance at 0x2ae0d78826c8>
('YRM1_Mnn4pr', 0)
<analyze_sig_hits_v2.pssm instance at 0x2ae0d7882170>
('YRM1_Tpo4/Mod5pr', 0)
<analyze_sig_hits_v2.pssm instance at 0x2ae0d7882c68>
('YRM1_control_Hsv2pr', 0)
<analyze_sig_hits_v2.pssm instance at 0x2ae0d7882f38>
('YRM1_control_Cax4pr', 0)
<analyze_sig_hits_v2.pssm instance at 0x2ae0d78826c8>
('TEA1_Vma13pr', 0)
<analyze_sig_hits_v2.pssm instance at 0x2ae0d7882998>
('TEA1_Rg12/Vhr1pr', 0)
<analyze_sig_hits_v2.pssm instance at 0x2ae0d7882c68>
('TEA1_Gln1pr', 0)
<analyze_sig_hits

('LEU3_Ipt1/Snf11pr', 0)
<analyze_sig_hits_v2.pssm instance at 0x2ae0d78826c8>
('LEU3_Mdh2pr', 0)
<analyze_sig_hits_v2.pssm instance at 0x2ae0d7882998>
('LEU3_Sod1/Ado1pr', 0)
<analyze_sig_hits_v2.pssm instance at 0x2ae0d7882518>
('LEU3_control_YBL028C', 0)
<analyze_sig_hits_v2.pssm instance at 0x2ae0d7882c68>
('LEU3_control_Leu1pr', 0)
<analyze_sig_hits_v2.pssm instance at 0x2ae0d78826c8>
('LYS14_Lys12pr', 1)
<analyze_sig_hits_v2.pssm instance at 0x2ae0d7882998>
('LYS14_LYP1pr', 0)
<analyze_sig_hits_v2.pssm instance at 0x2ae0d7882518>
('LYS14_control_Lys9pr', 0)
<analyze_sig_hits_v2.pssm instance at 0x2ae0d7882c68>
('LY14_control_Lys1pr', 0)
<analyze_sig_hits_v2.pssm instance at 0x2ae0d78826c8>
('ADR1pr_reg1\xc2\xa0', 0)
<analyze_sig_hits_v2.pssm instance at 0x2ae0d7882f38>
('HAP4pr_reg1', 0)
<analyze_sig_hits_v2.pssm instance at 0x2ae0d7882518>
('STP4pr_reg1', 1)
<analyze_sig_hits_v2.pssm instance at 0x2ae0d7882998>
('GID8pr_reg1', 0)
<analyze_sig_hits_v2.pssm instance at 0x2ae0d7882

<analyze_sig_hits_v2.pssm instance at 0x2ae0d7882c68>
('YRM1_Tpo4/Mod5pr', 1)
<analyze_sig_hits_v2.pssm instance at 0x2ae0d78826c8>
('YRM1_control_Hsv2pr', 0)
<analyze_sig_hits_v2.pssm instance at 0x2ae0d7882f38>
('YRM1_control_Cax4pr', 0)
<analyze_sig_hits_v2.pssm instance at 0x2ae0d7882998>
('TEA1_Vma13pr', 0)
<analyze_sig_hits_v2.pssm instance at 0x2ae0d7882c68>
('TEA1_Rg12/Vhr1pr', 0)
<analyze_sig_hits_v2.pssm instance at 0x2ae0d78826c8>
('TEA1_Gln1pr', 0)
<analyze_sig_hits_v2.pssm instance at 0x2ae0d7882f38>
('TEA1_control_Isf1pr ', 0)
<analyze_sig_hits_v2.pssm instance at 0x2ae0d7882998>
('TEA1_control_Adh3pr', 0)
<analyze_sig_hits_v2.pssm instance at 0x2ae0d7882c68>
('LEU3_Ipt1/Snf11pr', 0)
<analyze_sig_hits_v2.pssm instance at 0x2ae0d7882170>
('LEU3_Mdh2pr', 0)
<analyze_sig_hits_v2.pssm instance at 0x2ae0d7882f38>
('LEU3_Sod1/Ado1pr', 0)
<analyze_sig_hits_v2.pssm instance at 0x2ae0d78826c8>
('LEU3_control_YBL028C', 0)
<analyze_sig_hits_v2.pssm instance at 0x2ae0d7882c68>
('LEU3